# Mercado de Energia 2024.3
Aluno: Gabriel Halfeld Limp

Professores: Bruno Dias e Thiago Soares

##Instalando Bibliotecas

In [2]:
!pip install  pyomo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.6 MB/s eta 0:00:00


In [3]:
## GLPK installation
!apt-get install -y -qq glpk-utils
# IPOPT Solver Installation
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123623 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

##Primeira Aula

###Problema do Transporte

####Formulação Matemática

Sabendo que: \\

$
\begin{align}
  & i = \text{origens} \\
  \\
  & j = \text{destinos} \\
  \\
  & cap_{i} = \text{suprimento do produto na origem} \\
  \\
  & dem_{j} = \text{demanda do produto no destino} \\
  \\
  & c_{ij} = \text{custo por unidade de transporte entre origem i e destino j} \\
  \\
  & x_{ij} = \text{variável correspondente à quantidade de produto transportado entre origem i e destino j} \\
\end{align}
$

\
\

$$
\text{ ...::: O problema de otimização pode ser modelado da seguinte forma :::...}
$$

\
\

\begin{align}
  & \text{min $C_{t}$} = \Sigma_{i=1}^{NumOrig} \Sigma_{j=1}^{NumDest} ~~~ c_{ij} \cdot x_{ij} \tag{1} \\ \\
  & \text{s.t.} \notag \\ \\
  & \Sigma_{j=1}^{NumDest} ~~~ x_{ij} \leq cap_{i} \hspace{0.45cm}\forall \hspace{0.25cm} i \tag{2} \\
  & \Sigma_{i=1}^{NumOrig} ~~~ x_{ij} \geq dem_{j} \hspace{0.45cm}\forall \hspace{0.25cm} j \tag{3} \\
  & x_{ij} \geq 0 \hspace{3.5cm} \forall \hspace{0.25cm} i,j \tag{4} \\
\end{align}

####Modelo do Problema

In [4]:
# Dados Entrada

i = ['seattle', 'san_diego']
j = ['new_york','chicago','topeka']

cap = [350,600]
dem = [325,300,275]

distancias = pd.DataFrame([[2.5,1.7,1.8],[2.5,1.8,1.4]], columns = j, index = i) # em 1000 km

preco_km = 90 # preco de transporte a cada 1000 km

custo = (preco_km * distancias)/1000   # custo de transporte por km, entre origens e destinos

In [5]:
ni = len(i)
nj = len(j)

model = ConcreteModel()

#Sets:
model.i = Set(initialize=i)
model.j = Set(initialize=j)

#Parâmetros
model.cap = Param(model.i, initialize={i[k]: cap[k] for k in range(ni)})
model.dem = Param(model.j, initialize={j[k]: dem[k] for k in range(nj)})
model.distancias = Param(model.i, model.j, initialize=lambda model, i, j: distancias.loc[i,j])
model.custo = Param(model.i, model.j, initialize=lambda model, i, j: custo.loc[i,j])

#Variáveis
model.x = Var(model.i, model.j, domain=NonNegativeReals)

#Declarar Função Objetivo
def objective_rule(model):
    return sum(model.custo[i,j]*model.x[i,j] for i in model.i for j in model.j)
model.objective = Objective(rule=objective_rule, sense=minimize)

#Declarar Restrições
def supply_rule(model,i):
    return sum(model.x[i,j] for j in model.j) <= model.cap[i]
model.supply = Constraint(model.i, rule=supply_rule)

def demand_rule(model,j):
    return sum(model.x[i,j] for i in model.i) >= model.dem[j]
model.demand = Constraint(model.j, rule=demand_rule)

#Executar Otimização
solver = SolverFactory('glpk')
resultados = solver.solve(model, tee=False)

####Resultados

In [6]:
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo:', value(model.objective), '\n')

for orig in model.i:
    for dest in model.j:
        print('Produtos transportados de {} para {}:'.format(orig, dest), model.x[orig, dest].value)

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: 153.675 

Produtos transportados de seattle para new_york: 50.0
Produtos transportados de seattle para chicago: 300.0
Produtos transportados de seattle para topeka: 0.0
Produtos transportados de san_diego para new_york: 275.0
Produtos transportados de san_diego para chicago: 0.0
Produtos transportados de san_diego para topeka: 275.0


###Despacho Econômico Linear

####Formulação Matemática

Sabendo que: \\

$
\begin{align}
  & i = \text{geradores} \\
  \\
  & Ng = \text{número de geradores} \\
  \\
  & dem = \text{demanda energética} \\
  \\
  & c_{i} = \text{custo de produção de energia do gerador i} \\
  \\
  & P_{max~i} = \text{potência máxima do gerador i} \\
  \\
  & P_{min~i} = \text{potência mínima do gerador j} \\
  \\
  & P_{ger~i} = \text{variável correspondente à potência gerada pelo gerador i} \\
\end{align}
$

\
\

$$
\text{ ...::: O problema de otimização pode ser modelado da seguinte forma :::...}
$$

\
\

\begin{align}
  & \text{min $C_{t}$} = \Sigma_{i=1}^{Ng} ~~~ c_{i} \cdot P_{ger~i} \tag{1} \\ \\
  & \text{s.t.} \notag \\ \\
  & \Sigma_{i=1}^{Ng} ~ P_{ger~i} = dem \tag{2} \\
  & P_{min~i} \leq P_{ger~i} \leq P_{max~i} \tag{3} \\
\end{align}

####Modelo do Problema

In [7]:
# Dados Entrada

ger = ['Gerador_1', 'Gerador_2', 'Gerador_3']
Ng = len(ger)
pmin = [120,30,80]
pmax = [530,180,250]
c_ener = [100,150,300]

loads = ['Carga_1']
Nc = len(loads)
dem = [780]

#Modelo Concreto:
model = ConcreteModel()

#Sets:
model.ger = Set(initialize=ger)
model.loads = Set(initialize=loads)

# Parâmetros:
model.pmin = Param(model.ger, initialize={ger[k]: pmin[k] for k in range(Ng)})
model.pmax = Param(model.ger, initialize={ger[k]: pmax[k] for k in range(Ng)})
model.c_ener = Param(model.ger, initialize={ger[k]: c_ener[k] for k in range(Ng)})
model.dem = Param(model.loads, initialize={loads[k]: dem[k] for k in range(Nc)})

#Variáveis:
model.prod = Var(model.ger, domain = NonNegativeReals)

In [8]:
#Restrições:
def prod_min(model, gerador):
    return model.prod[gerador] >= model.pmin[gerador]
model.prod_min = Constraint(model.ger, rule=prod_min)

def prod_max(model, gerador):
    return model.prod[gerador] <= model.pmax[gerador]
model.prod_max = Constraint(model.ger, rule=prod_max)

def balanco(model):
    return sum(model.prod[i] for i in model.ger) == sum(model.dem[j] for j in model.loads)
model.balanco = Constraint(rule=balanco)

#Função Objetivo:
def objective_rule(model):
    return sum(model.c_ener[i]*model.prod[i] for i in model.ger)
model.objective = Objective(rule=objective_rule, sense=minimize)

#Executar Otimização:
solver = SolverFactory('glpk')
resultados = solver.solve(model, tee=False)

####Resultados

In [9]:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo: R$', value(model.objective), '\n')

for gerador in model.ger:
    print('Energia gerada do {}: {:.2f} MWh'.format(gerador, model.prod[gerador].value))

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: R$ 102500.0 

Energia gerada do Gerador_1: 530.00 MWh
Energia gerada do Gerador_2: 170.00 MWh
Energia gerada do Gerador_3: 80.00 MWh


###Despacho Econômico Quadrático

####Formulação Matemática

Sabendo que: \\

$
\begin{align}
  & i = \text{geradores} \\
  \\
  & Ng = \text{número de geradores} \\
  \\
  & dem = \text{demanda energética} \\
  \\
  & a_{i} = \text{coeficiente quadrático do custo de produção de energia do gerador i} \\
  \\
  & b_{i} = \text{coeficiente linear do custo de produção de energia do gerador i} \\
  \\
  & c_{i} = \text{custo base de produção de energia do gerador i} \\
  \\
  & P_{max~i} = \text{potência máxima do gerador i} \\
  \\
  & P_{min~i} = \text{potência mínima do gerador j} \\
  \\
  & P_{ger~i} = \text{variável correspondente à potência gerada pelo gerador i} \\
\end{align}
$

\
\

$$
\text{ ...::: O problema de otimização pode ser modelado da seguinte forma :::...}
$$

\
\

\begin{align}
  & \text{min $C_{t}$} = \Sigma_{i=1}^{Ng} ~~~ a_{i} \cdot P_{ger~i}^{2} + b_{i} \cdot P_{ger~i} + c_{i} \tag{1} \\ \\
  & \text{s.t.} \notag \\ \\
  & \Sigma_{i=1}^{Ng} ~ P_{ger~i} = dem \tag{2} \\
  & P_{min~i} \leq P_{ger~i} \leq P_{max~i} \tag{3} \\
\end{align}

In [10]:
#Dados de Entrada
ger = ['Gerador_1', 'Gerador_2', 'Gerador_3', 'Gerador_4', 'Gerador_5']
Ng = len(ger)
pmin = [28,90,68,76,19]
pmax = [206,284,189,266,53]
c_ener_a = [3,4.05,4.05,3.99,3.88]
c_ener_b = [20,18.07,15.55,19.21,26.18]
c_ener_c = [100,98.87,104.26,107.21,95.31]

loads = ['Carga_1']
Nc = len(loads)
dem = [780]

In [11]:
# Criar Modelo Concreto
model = ConcreteModel('despacho_quad')

#Sets
model.ger = Set(initialize=ger)
model.loads = Set(initialize=loads)

#Parâmetros
model.pmin = Param(model.ger, initialize={ger[k]: pmin[k] for k in range(Ng)})
model.pmax = Param(model.ger, initialize={ger[k]: pmax[k] for k in range(Ng)})
model.c_ener_a = Param(model.ger, initialize={ger[k]: c_ener_a[k] for k in range(Ng)})
model.c_ener_b = Param(model.ger, initialize={ger[k]: c_ener_b[k] for k in range(Ng)})
model.c_ener_c = Param(model.ger, initialize={ger[k]: c_ener_c[k] for k in range(Ng)})

model.dem = Param(model.loads, initialize={loads[k]: dem[k] for k in range(Nc)})

#Variáveis
model.prod = Var(model.ger, domain = NonNegativeReals)

In [12]:
#Restrições
def prod_min(model, gerador):
    return model.prod[gerador] >= model.pmin[gerador]
model.prod_min = Constraint(model.ger, rule=prod_min)

def prod_max(model, gerador):
    return model.prod[gerador] <= model.pmax[gerador]
model.prod_max = Constraint(model.ger, rule=prod_max)

def balanco(model):
    return sum(model.prod[i] for i in model.ger) == sum(model.dem[j] for j in model.loads)
model.balanco = Constraint(rule=balanco)

In [13]:
#Função Objetivo:
def objective_rule(model):
    return sum(model.c_ener_a[gerador]*model.prod[gerador]*model.prod[gerador] + model.c_ener_b[gerador]*model.prod[gerador] + model.c_ener_c[gerador]  for gerador in model.ger)
model.objective = Objective(rule=objective_rule, sense=minimize)

#Executar Otimização:
solver = SolverFactory('ipopt')
resultados = solver.solve(model, tee=False)

####Resultados

In [14]:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo: R$', value(model.objective), '\n')

for gerador in model.ger:
    print('Energia gerada do {}: {:.2f} MWh'.format(gerador, model.prod[gerador].value))


Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: R$ 518016.33374691044 

Energia gerada do Gerador_1: 206.00 MWh
Energia gerada do Gerador_2: 172.74 MWh
Energia gerada do Gerador_3: 173.06 MWh
Energia gerada do Gerador_4: 175.20 MWh
Energia gerada do Gerador_5: 53.00 MWh


##Otimização em Leilões de Energia

###Exercício 4.2 - Despacho Linear

####Modelo do Problema

In [15]:
#Dados do problema
dados_ger = pd.DataFrame({
    'Barra': [1, 1, 2, 2, 3, 3, 3, 5, 5, 4, 4],
    'P_min (MW)': [0] * 11,
    'P_max (MW)': [10, 20, 15, 40, 30, 40, 20, 10, 20, 60, 75],
    'Custo/MW': [5, 15, 6, 7, 6, 15, 25, 10, 15, 30, 18]
})


bus = [f'Barra {bar+1}' for bar in range(6)]

loads = ['Carga 1']
Nc = len(loads)
dem = [250]


ger = [f'Gerador {gerador+1}' for gerador in range(len(dados_ger['Barra']))]
Ng = len(ger)
c_ener = dados_ger['Custo/MW'].tolist()
pmin = [0] * Ng
pmax = [10, 20, 15, 40, 30, 40, 20, 10, 20, 60, 75]

#Localização do Gerador
ger_loc = pd.DataFrame(0, index=ger, columns=bus)
# Preencher a matriz ger_loc com 1 onde o gerador está localizado em uma barra
for idx, row in dados_ger.iterrows():
    gerador = f'Gerador {idx + 1}'  # Nome do índice para cada gerador
    barra = f'Barra {row["Barra"]}'
    ger_loc.at[gerador, barra] = 1

#Localização da Carga
loads_loc = pd.DataFrame(0, index=loads, columns=bus)
loads_loc.loc['Carga 1', 'Barra 6'] = 1

In [16]:
model = ConcreteModel()

#Sets
model.ger = Set(initialize=ger)
model.loads = Set(initialize=loads)
model.bus = Set(initialize=bus)

#Parâmetros
model.pmin = Param(model.ger, initialize={ger[k]: pmin[k] for k in range(Ng)})
model.pmax = Param(model.ger, initialize={ger[k]: pmax[k] for k in range(Ng)})
model.c_ener = Param(model.ger, initialize={ger[k]: c_ener[k] for k in range(Ng)})
model.dem = Param(model.loads, initialize={loads[k]: dem[k] for k in range(Nc)})
model.ger_loc = Param(model.ger, model.bus, initialize=lambda model, g, b: ger_loc.loc[g, b], default=0, mutable=False)
model.load_loc = Param(model.loads, model.bus, initialize=lambda model, l, b: loads_loc.loc[l, b], default=0, mutable=False)

#Variáveis
model.prod = Var(model.ger, domain = NonNegativeReals)

In [17]:
#Restrições
def prod_min(model, gerador):
    return model.prod[gerador] >= model.pmin[gerador]
model.prod_min = Constraint(model.ger, rule=prod_min)

def prod_max(model, gerador):
    return model.prod[gerador] <= model.pmax[gerador]
model.prod_max = Constraint(model.ger, rule=prod_max)

def balanco(model):
    return sum(model.prod[i] for i in model.ger) == sum(model.dem[j] for j in model.loads)
model.balanco = Constraint(rule=balanco)

In [18]:
#Função Objetivo:
def objective_rule(model):
    return sum(model.c_ener[i]*model.prod[i] for i in model.ger)
model.objective = Objective(rule=objective_rule, sense=minimize)

#Executar Otimização:
solver = SolverFactory('glpk')
resultados = solver.solve(model, tee=False)

####Resultados

In [19]:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo: R$', value(model.objective), '\n')

for gerador in model.ger:
    print('Energia gerada do {}: {:.2f} MWh'.format(gerador, model.prod[gerador].value))

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: R$ 3070.0 

Energia gerada do Gerador 1: 10.00 MWh
Energia gerada do Gerador 2: 20.00 MWh
Energia gerada do Gerador 3: 15.00 MWh
Energia gerada do Gerador 4: 40.00 MWh
Energia gerada do Gerador 5: 30.00 MWh
Energia gerada do Gerador 6: 40.00 MWh
Energia gerada do Gerador 7: 0.00 MWh
Energia gerada do Gerador 8: 10.00 MWh
Energia gerada do Gerador 9: 20.00 MWh
Energia gerada do Gerador 10: 0.00 MWh
Energia gerada do Gerador 11: 65.00 MWh


###Exercício 4.3 - Leilão - bids de compra e venda'

####Modelo do Problema

In [20]:
# ID da planilha
sheet_id = '1a2XaZct07CVfJwZSE6Ywl1VtblPurkgnVeebq94xaEg'
# IDs das abas
tab_ids = {'Offers':'0',
           'Bids':'2060141366'}

offers = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={tab_ids["Offers"]}')

bids = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={tab_ids["Bids"]}')

num_offers = len(offers)
num_bids = len(bids)

#Modelo Concreto
model = ConcreteModel()

#Set vendedores:
model.offers = Set(initialize=range(num_offers))

#Set compradores:
model.bids = Set(initialize=range(num_bids))

#Parâmetros dos vendedores:
model.selling_price = Param(model.offers, initialize={i: offers.loc[i, "$/MW"] for i in range(num_offers)})
model.selling_entity = Param(model.offers, initialize={i: offers.loc[i, "Entity"] for i in range(num_offers)})

#Parâmetros ds compradores:
model.buying_price = Param(model.bids, initialize={i: bids.loc[i, "$/MW"] for i in range(num_bids)})
model.buying_entity = Param(model.bids, initialize={i: bids.loc[i, "Entity"] for i in range(num_bids)})

#Variável de compra:
buying_max = bids['Load (MW)'].tolist()
buying_bounds = {i: (0, buying_max[i]) for i in range(num_bids)}
model.buy = Var(model.bids, bounds=buying_bounds, domain=Reals)

#Variável de venda:
selling_max = offers['P_max (MW)'].tolist()
selling_bounds = {i: (0, selling_max[i]) for i in range(num_offers)}
model.sell = Var(model.offers, bounds = selling_bounds, domain=Reals)

#Restrição:
model.restr_igualdade = ConstraintList()
model.restr_igualdade.add(expr=sum(model.buy[i] for i in model.bids) - sum(model.sell[i] for i in model.offers) == 0)

#FOB:
def fob(model):
    compra = sum(model.buy[i]*model.buying_price[i] for i in model.bids)
    venda = sum(model.sell[i]*model.selling_price[i] for i in model.offers)
    return compra-venda
model.fob = Objective(rule=fob, sense=maximize)
model.dual = Suffix(direction=Suffix.IMPORT)
solver = SolverFactory('glpk')
results = solver.solve(model, tee=False)


'Any'. The default domain for Param objects is 'Any'.  However, we
will be changing that default to 'Reals' in the future.  If you really
intend the domain of this Paramto be 'Any', you can suppress this
warning by explicitly specifying 'within=Any' to the Param
constructor.  (deprecated in 5.6.9, will be removed in (or after) 6.0)
(called from /usr/local/lib/python3.10/dist-packages/pyomo/core/base/indexed_component.py:714)
'Any'. The default domain for Param objects is 'Any'.  However, we
will be changing that default to 'Reals' in the future.  If you really
intend the domain of this Paramto be 'Any', you can suppress this
warning by explicitly specifying 'within=Any' to the Param
constructor.  (deprecated in 5.6.9, will be removed in (or after) 6.0)
(called from /usr/local/lib/python3.10/dist-packages/pyomo/core/base/indexed_component.py:714)


####Resultados

In [21]:
#Exibir os resultados:
print('Status Final do Problema de Otimização:', results.solver.status, '\n')
print('Condição de Término:', results.solver.termination_condition, '\n')
print('Resultado Função Objetivo: R$', round(value(model.fob),2), '\n')
print('Resultados das Energias Vendidas:')
for i in model.offers:
    print(f'Lance {i+1} - Entidade {model.selling_entity[i]}: {model.sell[i].value} MWh')

print('Resultados das Energias Compradas:')
for i in model.bids:
    print(f'Lance {i+1} - Entidade {model.buying_entity[i]}: {model.buy[i].value} MWh')

# Obter o custo marginal (multiplicador de Lagrange) da restrição de igualdade
custo_marginal = model.dual[model.restr_igualdade[1]]
print("Custo Marginal: R$", custo_marginal)


Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: R$ 2430.0 

Resultados das Energias Vendidas:
Lance 1 - Entidade A: 20.0 MWh
Lance 2 - Entidade A: 40.0 MWh
Lance 3 - Entidade A: 0.0 MWh
Lance 4 - Entidade B: 0.0 MWh
Lance 5 - Entidade B: 10.0 MWh
Lance 6 - Entidade B: 0.0 MWh
Resultados das Energias Compradas:
Lance 1 - Entidade X: 0.0 MWh
Lance 2 - Entidade X: 20.0 MWh
Lance 3 - Entidade X: 5.0 MWh
Lance 4 - Entidade Y: 0.0 MWh
Lance 5 - Entidade Y: 30.0 MWh
Lance 6 - Entidade Y: 15.0 MWh
Custo Marginal: R$ 25.0


###Exercício 4.5 - Leilão com Rede

####Modelo do Problema

In [22]:
# ID da planilha
sheet_id = '1s9AzlV2unNyPs5EM3kBXp61iMpe6p3cO0xNeTXEOVLI'
# IDs das abas
tab_ids = {'Offers':'0',
           'Bids':'1696916279',
           'SF': '2136104990',
           'Network': '1354044027'}
offers = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={tab_ids["Offers"]}')

bids = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={tab_ids["Bids"]}')

SF = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={tab_ids["SF"]}', header=None)
SF = SF.replace(',', '.', regex=True).astype(float)
SF = np.array(SF)

network = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={tab_ids["Network"]}')

In [23]:
num_offers = len(offers)
num_bids = len(bids)
num_lines = len(network)
unique_locations = set(offers['Barra']).union(set(bids['Barra']))
num_nodes = len(unique_locations)

#Modelo Concreto
model = ConcreteModel()

#Set das Barras do Sistema
model.barras = Set(initialize=sorted(unique_locations))

#Set Linhas do Sistema
linhas = [f'Linha {i+1}' for i in range(num_lines)]
model.linhas = Set(initialize=linhas)

#Set Lances Vendedores
lances_vendedores = [f'Lance {i+1}' for i in range(num_offers)]
model.offers = Set(initialize=lances_vendedores)

#Set Lances Compradores
lances_compradores = [f'Lance {i+1}' for i in range(num_bids)]
model.bids = Set(initialize=lances_compradores)

#Parâmetros de Linha
barra_de = network['Barra de'].tolist()
model.barra_de = Param(model.linhas, initialize={linhas[i]: barra_de[i] for i in range(num_lines)}, within=model.barras)

barra_para = network['Barra para'].tolist()
model.barra_para = Param(model.linhas, initialize={linhas[i]: barra_para[i] for i in range(num_lines)}, within=model.barras)

fmax = np.array(network['F_max'].tolist())
model.fmax = Param(model.linhas, initialize={linhas[i]: network.loc[i, "F_max"] for i in range(num_lines)})
model.fmax.pprint()

#Parâmetros dos Lances Compradores
offers_bar = offers['Barra'].tolist()
model.offers_bar = Param(model.offers, initialize={lances_vendedores[i]: offers_bar[i] for i in range(num_offers)}, within=model.barras)
model.offers_bar.pprint()

offers_price = offers['Price'].tolist()
model.offers_price = Param(model.offers, initialize={lances_vendedores[i]: offers_price[i] for i in range(num_offers)})
model.offers_price.pprint()

#Parâmetros dos Lances Vendedores
bids_bar = bids['Barra'].tolist()
model.bids_bar = Param(model.bids, initialize={lances_compradores[i]: bids_bar[i] for i in range(num_bids)}, within=model.barras)
model.bids_bar.pprint()

bids_price = bids['Price'].tolist()
model.bids_price = Param(model.bids, initialize={lances_compradores[i]: bids_price[i] for i in range(num_bids)})
model.bids_price.pprint()

#Limite das variáveis
selling_max = offers['Qty'].tolist()
selling_bounds = {lances_vendedores[i]: (0, selling_max[i]) for i in range(num_offers)}

buying_max = bids['Qty'].tolist()
buying_bounds = {lances_compradores[i]: (0, buying_max[i]) for i in range(num_bids)}

#Variáveis de potência vendida de cada lance vendedor
model.sell_qty = Var(model.offers, bounds=selling_bounds, domain=Reals)
model.sell_qty.display()

#Variáveis de potência comprada de cada lance comprador
model.buy_qty = Var(model.bids, bounds=buying_bounds, domain=Reals)
model.buy_qty.display()

#Declarando Função Objetivo
def FOB(model):
    receita = sum(model.sell_qty[i]*model.offers_price[i] for i in model.offers)
    despesa = sum(model.buy_qty[j]*model.bids_price[j] for j in model.bids)
    return despesa-receita # Maximizar a diferença entre área compradora e vendedora

model.custo_total = Objective(rule=FOB, sense=maximize)

def potencia_injetada(model):
    liq_injection = []
    for barra in model.barras:
        pot_vendida = sum(model.sell_qty[i] for i in model.offers if model.offers_bar[i] == barra)
        pot_comprada = sum(model.buy_qty[j] for j in model.bids if model.bids_bar[j] == barra)
        pot_injetada = pot_vendida - pot_comprada
        liq_injection.append(pot_injetada)
    return liq_injection

def fluxo(model, SF):
    # Função para calcular o fluxo em cada linha como uma expressão Pyomo
    pot_inj = np.array(potencia_injetada(model))
    return np.dot(SF, pot_inj)

#Restrição de Fluxo:
model.restr_desigualdade = ConstraintList()
for i in range(num_lines):
    model.restr_desigualdade.add(fluxo(model, SF)[i] >= -fmax[i])
    model.restr_desigualdade.add(fluxo(model, SF)[i] <= fmax[i])
model.restr_desigualdade.pprint()

#Balanço de Potência do Sistema
model.restr_igualdade = ConstraintList()
model.restr_igualdade.add(expr=sum(model.buy_qty[i] for i in model.bids) - sum(model.sell_qty[i] for i in model.offers) == 0)

solver = SolverFactory('glpk')
# Adicionar o componente para armazenar duais
model.dual = Suffix(direction=Suffix.IMPORT)

results = solver.solve(model, tee=False)

fmax : Size=6, Index=linhas, Domain=Any, Default=None, Mutable=False
    Key     : Value
    Linha 1 :   100
    Linha 2 :   100
    Linha 3 :   100
    Linha 4 :   100
    Linha 5 :   100
    Linha 6 :   100
offers_bar : Size=11, Index=offers, Domain=barras, Default=None, Mutable=False
    Key      : Value
     Lance 1 :     A
    Lance 10 :     D
    Lance 11 :     D
     Lance 2 :     A
     Lance 3 :     B
     Lance 4 :     B
     Lance 5 :     C
     Lance 6 :     C
     Lance 7 :     C
     Lance 8 :     D
     Lance 9 :     D
offers_price : Size=11, Index=offers, Domain=Any, Default=None, Mutable=False
    Key      : Value
     Lance 1 :     5
    Lance 10 :    30
    Lance 11 :    18
     Lance 2 :    15
     Lance 3 :     6
     Lance 4 :     7
     Lance 5 :     6
     Lance 6 :    15
     Lance 7 :    25
     Lance 8 :    10
     Lance 9 :    15
bids_bar : Size=7, Index=bids, Domain=barras, Default=None, Mutable=False
    Key     : Value
    Lance 1 :     E
    Lance 2 :   

####Resultados

In [24]:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', results.solver.status, '\n')
print('Condição de Término:', results.solver.termination_condition, '\n')
print('Resultado Função Objetivo:', value(model.custo_total), '\n')

# Exibindo as variáveis de potência vendida de cada lance vendedor
print("Resultados das Potências Vendidas (Lances dos Vendedores):")
venda_total = 0
for lance in model.offers:
    venda_total += model.sell_qty[lance].value*model.offers_price[lance]
    print(f"{lance}: {model.sell_qty[lance].value} MWh")
print(f'Venda total:{venda_total}$')

# Exibindo as variáveis de potência comprada de cada lance comprador
compra_total = 0
print("\nResultados das Potências Compradas (Lances dos Compradores):")
for lance in model.bids:
    compra_total += model.buy_qty[lance].value*model.bids_price[lance]
    print(f"{lance}: {model.buy_qty[lance].value} MWh")
print(f'Compra total: R${compra_total}')

# Obter o custo marginal (multiplicador de Lagrange) da restrição de igualdade
custo_marginal = model.dual[model.restr_igualdade[1]]
print("Custo Marginal: R$", custo_marginal)

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: 4275.0 

Resultados das Potências Vendidas (Lances dos Vendedores):
Lance 1: 10.0 MWh
Lance 2: 20.0 MWh
Lance 3: 15.0 MWh
Lance 4: 40.0 MWh
Lance 5: 30.0 MWh
Lance 6: 35.0 MWh
Lance 7: 0.0 MWh
Lance 8: 10.0 MWh
Lance 9: 20.0 MWh
Lance 10: 0.0 MWh
Lance 11: 0.0 MWh
Venda total:1825.0$

Resultados das Potências Compradas (Lances dos Compradores):
Lance 1: 20.0 MWh
Lance 2: 0.0 MWh
Lance 3: 25.0 MWh
Lance 4: 45.0 MWh
Lance 5: 50.0 MWh
Lance 6: 0.0 MWh
Lance 7: 40.0 MWh
Compra total: R$6100.0
Custo Marginal: R$ 15.0


###Exercício Aula 6 - Slide 29

####Modelo do Problema

In [25]:
bus = ['Barra 1', 'Barra 2', 'Barra 3']
Nb = len(bus)

ger = ['Gerador_1', 'Gerador_2']
Ng = len(ger)
p_min = [0, 0]
p_max = [390, 9999]
C_ener = [10, 20]

loads = ['Carga_1']
Nc = len(loads)
dem = [450]

ger_loc = pd.DataFrame([[1,0,0], [0,1,0]], columns=bus, index=ger)
loads_loc = pd.DataFrame([[0,0,1]], columns=bus, index=loads)

conex = pd.DataFrame([[0,1,1],[1,0,1],[1,1,0]], columns=bus, index=bus) #Matriz de conexões
x_line = pd.DataFrame([[0,1,1],[1,0,1],[1,1,0]], columns=bus, index=bus) #Matriz de reatâncias
t_line = pd.DataFrame([[0,200,260],[200,0,200],[260,200,0]], columns=bus, index=bus) #Matriz de limites térmicos das linhas

In [26]:
model = ConcreteModel()

#Geradores Térmicos:
model.ger = Set(initialize=ger)
model.c_ener = Param(model.ger, initialize={ger[i]: C_ener[i] for i in range(Ng)})
model.pmax_ter = Param(model.ger, initialize={ger[i]: p_max[i] for i in range(Ng)})
model.pmin_ter = Param(model.ger, initialize={ger[i]: p_min[i] for i in range(Ng)})

#Cargas
model.loads = Set(initialize=loads)
model.dem = Param(model.loads, initialize={loads[i]: dem[i] for i in range(Nc)})

#Barras e Localizações
model.bus = Set(initialize=bus)
model.ger_loc = Param(model.ger, model.bus, initialize=lambda model, g, b: ger_loc.loc[g, b], default=0, mutable=False)
model.loads_loc = Param(model.loads, model.bus, initialize=lambda model, l, b: loads_loc.loc[l, b], default=0, mutable=False)
model.conex = Param(model.bus, model.bus, initialize=lambda model, b1, b2: conex.loc[b1, b2], default=0, mutable=False)
model.x_line = Param(model.bus, model.bus, initialize=lambda model, b1, b2: x_line.loc[b1, b2], default=0, mutable=False)
model.t_line = Param(model.bus, model.bus, initialize=lambda model, b1, b2: t_line.loc[b1, b2], default=0, mutable=False)

In [27]:
# Declarar Variáveis
model.P_ger = Var(model.ger, domain=NonNegativeReals) # Produção dos Geradores Térmicos
model.theta = Var(model.bus, domain=Reals)

In [28]:
#Restrições de Limites:
def prod_min(model, gerador):
    return model.P_ger[gerador] >= model.pmin_ter[gerador]
model.prod_min = Constraint(model.ger, rule=prod_min)

def prod_max(model, gerador):
    return model.P_ger[gerador] <= model.pmax_ter[gerador]
model.prod_max = Constraint(model.ger, rule=prod_max)

def flow_limit(model, barra1, barra2):
    if model.conex[barra1, barra2] == 1:
        return (model.theta[barra1] - model.theta[barra2])/model.x_line[barra1,barra2] <= model.t_line[barra1,barra2]
    else:
        return Constraint.Skip
model.flow_limit = Constraint(model.bus, model.bus, rule=flow_limit)

#Restrição de Balanço
def balanco(model, bus):
    thermal_generation = sum(model.P_ger[g]*model.ger_loc[g,bus] for g in model.ger)
    load_demand = sum(model.dem[l]*model.loads_loc[l,bus] for l in model.loads)
    line_flow = sum((model.theta[bus] - model.theta[bar])/model.x_line[bus, bar] for bar in model.bus if model.conex[bus,bar]==1)
    return thermal_generation - load_demand == line_flow
model.balanco = Constraint(model.bus, rule=balanco)

In [29]:
#Função Objetivo
def objective_rule(model):
    return sum(model.c_ener[i]*model.P_ger[i] for i in model.ger)
model.objective = Objective(rule=objective_rule, sense=minimize)

In [30]:
#Otimizando:
solver = SolverFactory('glpk')
resultados = solver.solve(model, tee=False)

####Resultados

In [31]:
#Relatório de Resultados
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo: R$', value(model.objective), '\n')

for gerador in model.ger:
    print('Energia gerada do {}: {:.2f} MWh'.format(gerador, model.P_ger[gerador].value))

for barra1 in model.bus:
    for barra2 in model.bus:
        if model.conex[barra1, barra2] == 1:
            flux = (model.theta[barra1].value - model.theta[barra2].value)/model.x_line[barra1, barra2]
            print(f'Fluxo de {barra1} para {barra2}: {flux}')

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: R$ 5700.0 

Energia gerada do Gerador_1: 330.00 MWh
Energia gerada do Gerador_2: 120.00 MWh
Fluxo de Barra 1 para Barra 2: 70.0
Fluxo de Barra 1 para Barra 3: 260.0
Fluxo de Barra 2 para Barra 1: -70.0
Fluxo de Barra 2 para Barra 3: 190.0
Fluxo de Barra 3 para Barra 1: -260.0
Fluxo de Barra 3 para Barra 2: -190.0


##Lista Despacho Econômico com Rede

####Modelo do Problema

###Exercício 1
As funções de custo total para os dois geradores são

C1(g1) = 5 + 4g1 + g1^2

C2(g2) = 5 + 2g2 + 2g2^2

A demanda total de eletricidade é de 30 MWh.
Resolva o problema de despacho econômico, determinando g1, g2, λ, custo
de cada gerador e custo total do sistema.

In [38]:
numger = 2;
numload = 1;
ger = [f'Gerador {i}' for i in range(1, numger + 1)]
custo_a = [1, 2]
custo_b = [4, 2]
custo_c = [5, 5]
carga = [f'Carga {i}' for i in range(1, numload + 1)]
demanda = 30

#Modelo Concreto
model = ConcreteModel()
model.ger = Set(initialize=ger)
model.carga = Set(initialize=carga)

# Parâmetros
model.a = Param(model.ger, initialize={ger[i]: custo_a[i] for i in range(numger)})
model.b = Param(model.ger, initialize={ger[i]: custo_b[i] for i in range(numger)})
model.c = Param(model.ger, initialize={ger[i]: custo_c[i] for i in range(numger)})
model.demanda = Param(model.carga, initialize={carga[i]: demanda for i in range(numload)})

#Variáveis
model.pot = Var(model.ger, domain=NonNegativeReals)

#Restrição
model.restr = Constraint(expr = sum(model.pot[i] for i in model.ger) - sum(model.demanda[i] for i in model.carga) == 0)

#FOB
def FOB(model):
    custo = sum(model.a[i]*model.pot[i]**2 + model.b[i]*model.pot[i] + model.c[i] for i in model.ger)
    return custo
model.FOB = Objective(rule=FOB, sense=minimize)

# Adicionar o componente para armazenar duais
model.dual = Suffix(direction=Suffix.IMPORT)

#Solver
solver = SolverFactory('ipopt')
solver.options['output_file'] = 'ipopt.out'
solver.options['dual_inf_tol'] = 1e-6
results = solver.solve(model, tee=False)

####Resultados

In [40]:
#Resultados:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', results.solver.status, '\n')
print('Condição de Término:', results.solver.termination_condition, '\n')
print('Resultado Função Objetivo:', value(model.FOB), '\n')

# Exibindo as variáveis de potência vendida de cada lance vendedor
print("Resultados das Potências Vendidas (Lances dos Vendedores):")
venda_total = 0
for gerador in model.ger:
    venda_total += model.a[gerador]*model.pot[gerador].value**2 + model.b[gerador]*model.pot[gerador].value + model.c[gerador]
    print(f"{gerador}: {model.pot[gerador].value} MWh")
    print(f"Preço do {gerador}: R${model.a[gerador]*model.pot[gerador].value**2 + model.b[gerador]*model.pot[gerador].value + model.c[gerador]}")
print(f'Venda total:{venda_total}$')

# Verificação e impressão do custo marginal
if model.restr in model.dual:
    custo_marginal = model.dual[model.restr]
    print("Custo Marginal: R$", custo_marginal)
else:
    print("Valor dual da restrição não está disponível.")

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: 709.6666666666667 

Resultados das Potências Vendidas (Lances dos Vendedores):
Gerador 1: 19.666666666647487 MWh
Preço do Gerador 1: R$470.4444444436133
Gerador 2: 10.333333333352515 MWh
Preço do Gerador 2: R$239.22222222305342
Venda total:709.6666666666667$
Custo Marginal: R$ 43.33333333316758


##Programação Estocástica - Sequencial

###Modelo do Problema

In [41]:
# Dados Entrada
g = ['Produtor A', 'Produtor B']
cap_max = [100,100] # MW
ng = len(g)

scen = ['Cenario 1', 'Cenario 2']
prob_cenario = [0.05,0.95] #Aqui modelei probabilidade de cenario pra diferentes cargas, basta adicionar uma nova linha
demanda_reserva = [20,10] # por cenario
ns = len(scen)

consumidor = ['Consumidor 1']
nc = len(consumidor)
demanda_energia = [130]

preco_energia = [10,30] # $/MWh
preco_reserva = [0,25]  # $/MW

corte = ['Corte de Carga 1']  #Aqui também ficou possível adicionar cortes diferentes com preços diferentes
ncorte = len(corte)
preco_corte = [100]      # $/MWh

# Criar Modelo Concreto - Mercado de Energia
model_energia = ConcreteModel()

model_energia.ger = Set(initialize=g)
model_energia.price = Param(model_energia.ger, initialize={g[i]: preco_energia[i] for i in range(ng)})
model_energia.pmax = Param(model_energia.ger, initialize={g[i]: cap_max[i] for i in range(ng)})

model_energia.carga = Set(initialize=consumidor)
model_energia.dem = Param(model_energia.carga, initialize={consumidor[i]: demanda_energia[i] for i in range(nc)})

#Criar Modelo Concreto - Mercado de Reserva com Cenários Probabilísticos
model_reserva = ConcreteModel()

model_reserva.ger = Set(initialize=g)
model_reserva.price = Param(model_reserva.ger, initialize={g[i]: preco_energia[i] for i in range(ng)})
model_reserva.price_res = Param(model_reserva.ger, initialize={g[i]: preco_reserva[i] for i in range(ng)})
model_reserva.pmax = Param(model_reserva.ger, initialize={g[i]: cap_max[i] for i in range(ng)})

model_reserva.carga = Set(initialize=consumidor)
model_reserva.cenario = Set(initialize=scen)
model_reserva.dem_res = Param(model_reserva.cenario, initialize={scen[j]: demanda_reserva[j] for j in range(ns)})
model_reserva.prob = Param(model_reserva.cenario, initialize={scen[j]: prob_cenario[j] for j in range(ns)})

model_reserva.corte = Set(initialize=corte)
model_reserva.price_cut = Param(model_reserva.corte, initialize={corte[i]: preco_corte[i] for i in range(ncorte)})

# Declarar Variáveis - Mercado de Energia
model_energia.P = Var(model_energia.ger, domain=NonNegativeReals)

# Declarar Variáveis - Mercado de Reserva
model_reserva.R = Var(model_reserva.ger, domain=NonNegativeReals)
model_reserva.r = Var(model_reserva.ger, model_reserva.cenario, domain=NonNegativeReals)
model_reserva.L = Var(model_reserva.corte, model_reserva.cenario, domain = NonNegativeReals)

# Funções Objetivo

#FOB do Mercado de Energia
def FOB_energia(model):
    #Parte Determinística
    FOB = sum(model.price[g] * model.P[g]  for g in model.ger)
    return FOB
model_energia.objective = Objective(rule=FOB_energia(model_energia), sense=minimize)

#FOB do Mercado de Reserva
def FOB_reserva(model):
    #Parte Determinística
    FOB_det = sum(model.price_res[g] * model.R[g]  for g in model.ger)
    #Parte Probabilística
    FOB_prob = 0
    for i in model.cenario:
        FOB_prob += model.prob[i] * (sum(model.price[g] * model.r[g,i] for g in model.ger)
                                    +sum(model.price_cut[j] * model.L[j,i] for j in model.corte))
    return FOB_det + FOB_prob

model_reserva.objective = Objective(rule=FOB_reserva(model_reserva), sense=minimize)

#Restrições do Mercado de Energia
model_energia.balanco = Constraint(expr=sum(model_energia.P[g] for g in model_energia.ger) == sum(model_energia.dem[j] for j in model_energia.carga))

model_energia.limites = ConstraintList()
for g in model_energia.ger:
    model_energia.limites.add(expr=model_energia.P[g] <= model_energia.pmax[g])

solver = SolverFactory('glpk')
# solver = SolverFactory('ipopt',executable='/content/ipopt')

mercado_energia = solver.solve(model_energia, tee=False)


###Resultados

In [42]:
#Print das Variáveis
print(f'FOB do Mercado de Energia:' ,value(model_energia.objective))
print('Status Final do Problema de Otimização:', mercado_energia.solver.status, '\n')
for i in model_energia.ger:
    print(f'Produção do {i}:' ,value(model_energia.P[i]))

#Restrições do Mercado de Reserva
model_reserva.restr = ConstraintList()
for g in model_reserva.ger:
    model_reserva.restr.add(expr=model_reserva.R[g] + value(model_energia.P[g]) <= model_reserva.pmax[g])  ##Reparar que aqui essa restrição do mercado de reserva depende de uma variável do mercado de energia: Resolver nessa ordem!!
    for i in model_reserva.cenario:
        model_reserva.restr.add(expr=model_reserva.r[g,i] <= model_reserva.R[g])
for i in model_reserva.cenario:
    model_reserva.restr.add(expr=sum(model_reserva.r[g,i] for g in model_reserva.ger) + sum(model_reserva.L[j,i] for j in model_reserva.corte) == model_reserva.dem_res[i])
    model_reserva.restr.add(expr=sum(model_reserva.L[j,i] for j in model_reserva.corte) <= model_reserva.dem_res[i])
mercado_reserva = solver.solve(model_reserva, tee=False)

print(f'\n FOB do Mercado de Reserva:' ,value(model_reserva.objective))

for j in model_reserva.ger:
    print(f'Reserva do {j}:' ,value(model_reserva.R[j]))
    print(f'Custo da Reserva do {j}:' ,value(model_reserva.R[j])*model_reserva.price_res[j])


for i in model_reserva.cenario:
    for j in model_reserva.ger:
        print(f'\nReserva ativada do {j} no {i}:' ,value(model_reserva.r[j,i]))
        print(f'Custo da Reserva Ativada do {j} no {i}:' ,value(model_reserva.r[j,i])*model_reserva.price[j])
    for j in model_reserva.corte:
        print(f'\nCorte de Carga do {i}:' ,value(model_reserva.L[j,i]))
        print(f'Custo do Corte de Carga do {i}:' ,value(model_reserva.L[j,i])*model_reserva.price_cut[j])

FOB do Mercado de Energia: 1900.0
Status Final do Problema de Otimização: ok 

Produção do Produtor A: 100.0
Produção do Produtor B: 30.0

 FOB do Mercado de Reserva: 600.0
Reserva do Produtor A: 0.0
Custo da Reserva do Produtor A: 0.0
Reserva do Produtor B: 10.0
Custo da Reserva do Produtor B: 250.0

Reserva ativada do Produtor A no Cenario 1: 0.0
Custo da Reserva Ativada do Produtor A no Cenario 1: 0.0

Reserva ativada do Produtor B no Cenario 1: 10.0
Custo da Reserva Ativada do Produtor B no Cenario 1: 300.0

Corte de Carga do Cenario 1: 10.0
Custo do Corte de Carga do Cenario 1: 1000.0

Reserva ativada do Produtor A no Cenario 2: 0.0
Custo da Reserva Ativada do Produtor A no Cenario 2: 0.0

Reserva ativada do Produtor B no Cenario 2: 10.0
Custo da Reserva Ativada do Produtor B no Cenario 2: 300.0

Corte de Carga do Cenario 2: 0.0
Custo do Corte de Carga do Cenario 2: 0.0


##Programação Estocástica - Conjunto

###Demanda Incerta

####Modelo do Problema

In [43]:
# Dados Entrada

ger = ['Produtor A', 'Produtor B']
Ng = len(ger)
pmax_ter = [100,100] # MW

loads = ['Consumidor 1']
Nc = len(loads)
demanda_energia = [130]

scen = ['Cenario_1', 'Cenario_2']
Ns = len(scen)
prob_cenario = [0.05,0.95] # probabilidade de ocorência dos cenários
dem_reserva_real = pd.DataFrame([[20,10]], columns = scen, index = loads) # Carga por cenários
dem_reserva_forecast = np.dot(dem_reserva_real, prob_cenario) # Forecast da Carga


C_ener = [10,30] # $/MWh
C_res = [0,25]  # $/MW
C_shed = [100]  # $/MWh por carga

# Criar Modelo Concreto
model = ConcreteModel()

#Geradores Térmicos:
model.ger = Set(initialize=ger)
model.c_ener = Param(model.ger, initialize={ger[i]: C_ener[i] for i in range(Ng)})
model.c_res = Param(model.ger, initialize={ger[i]: C_res[i] for i in range(Ng)})
model.pmax_ter = Param(model.ger, initialize={ger[i]: pmax_ter[i] for i in range(Ng)})

#Cargas:
model.loads = Set(initialize=loads)
model.dem = Param(model.loads, initialize={loads[i]: demanda_energia[i] for i in range(Nc)})
model.c_shed = Param(model.loads, initialize={loads[i]: C_shed[i] for i in range(Nc)})

#Cenários:
model.scen = Set(initialize=scen)
model.prob = Param(model.scen, initialize={scen[i]: prob_cenario[i] for i in range(Ns)})
dem_res_real = {(loads[i], scen[j]): dem_reserva_real.loc[loads[i], scen[j]] for i in range(Nc) for j in range(Ns)}
model.dem_res_real = Param(model.loads, model.scen, initialize=dem_res_real)
dem_res_forecast = Param(model.loads, initialize={loads[i]: dem_reserva_forecast[i] for i in range(Nc)})

# Declarar Variáveis
model.P_ger = Var(model.ger, domain=NonNegativeReals)
model.R = Var(model.ger, domain=NonNegativeReals)
model.r = Var(model.ger, model.scen, domain=NonNegativeReals)
model.L_shed = Var(model.loads, model.scen, domain = NonNegativeReals)

#Restrições
def day_ahead_balance(model): #Restrição do balanço de energia no mercado diário
    thermal_generation =  sum(model.P_ger[g] for g in model.ger)
    demand = sum(model.dem[l] for l in model.loads)
    return thermal_generation == demand
model.day_ahead_balance = Constraint(rule=day_ahead_balance)

def upper_bounds(model, ger): # restrição 3: limite de geração
    return model.P_ger[ger] + model.R[ger] <= model.pmax_ter[ger]
model.UpperBound = Constraint(model.ger, rule=upper_bounds)

def non_ant_res(model, ger, cen):
    return model.r[ger,cen] <= model.R[ger]
model.NonAntRes = Constraint(model.ger, model.scen, rule=non_ant_res)

def operation_balance(model, carga, cen): #Restrição do balanço
    thermal_reserve_ativation = sum(model.r[g,cen] for g in model.ger)
    shed_load = sum(model.L_shed[l,cen] for l in model.loads)
    return thermal_reserve_ativation + shed_load == model.dem_res_real[carga,cen]
model.operation_balance = Constraint(model.loads, model.scen, rule=operation_balance)

def non_ant_shed(model, carga, scen): # restrição non-anticipativity corte carga (load,scen)
    return model.L_shed[carga,scen] <= model.dem_res_real[carga,scen]
model.NonAntShed = Constraint(model.loads, model.scen, rule=non_ant_shed)

# Declarar Função Objetivo
def FOB(model):
    #Parte Determinística
    custo_energia = sum(model.c_ener[g]*model.P_ger[g] for g in model.ger)
    custo_reserva = sum(model.c_res[g]*model.R[g] for g in model.ger)
    FOB_det = custo_energia + custo_reserva

    #Parte Probabilística
    FOB_prob = 0
    for cen in model.scen:
        custo_reserva_ativada = sum(model.c_ener[g]*model.r[g,cen] for g in model.ger)
        custo_deslastre = sum(model.c_shed[l]*model.L_shed[l,cen] for l in model.loads)
        FOB_prob += model.prob[cen] * (custo_reserva_ativada + custo_deslastre)
    return FOB_det + FOB_prob

model.objective = Objective(rule=FOB, sense=minimize)

solver = SolverFactory('ipopt')
resultados = solver.solve(model, tee=False)

####Resultados

In [44]:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo: $', f"{value(model.objective):.2f}", '\n')

# Print das Variáveis
for i in model.ger:
    print(f'Produção do {i}:', f"{value(model.P_ger[i]):.2f}", ' MWh\n')
    print(f'Reserva do {i}:', f"{value(model.R[i]):.2f}", 'MW\n')

for j in model.scen:
    for i in model.loads:
        print(f'Deslastre do {i} no {j}:', f"{value(model.L_shed[i, j]):.2f}", 'MWh \n')
    for i in model.ger:
        print(f'Reserva ativada do {i} no {j}:', f"{value(model.r[i, j]):.2f}", 'MW \n')

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: $ 2250.00 

Produção do Produtor A: 90.00  MWh

Reserva do Produtor A: 10.00 MW

Produção do Produtor B: 40.00  MWh

Reserva do Produtor B: 0.00 MW

Deslastre do Consumidor 1 no Cenario_1: 10.00 MWh 

Reserva ativada do Produtor A no Cenario_1: 10.00 MW 

Reserva ativada do Produtor B no Cenario_1: 0.00 MW 

Deslastre do Consumidor 1 no Cenario_2: 0.00 MWh 

Reserva ativada do Produtor A no Cenario_2: 10.00 MW 

Reserva ativada do Produtor B no Cenario_2: 0.00 MW 



###Uma Eólica

####Modelo do Problema

In [45]:
# Dados Entrada

ger = ['Termico_1', 'Termico_2', 'Termico_3']
Ng = len(ger)

wnd = ['Eolico_1']
Nw = len(wnd)

loads = ['Carga_1', 'Carga_2']
Nc = len(loads)

scen = ['Cenario_1', 'Cenario_2']
Ns = len(scen)

bus = ['Barra_1', 'Barra_2']
Nb = len(bus)

demanda_energia = [40,100] # por carga

C_ener = [10,30,35] # $/MWh por gerador térmico
C_up = [16,13,10]   # $/MWh por gerador térmico
C_dw = [15,12,9]    # $/MWh por gerador térmico
C_shed = [200,200]  # $/MWh por carga
C_wind = [0]        # $/MWh por gerador eólico

pmax_ter = [50,110,100] # potencia máxima geradores térmicos
pmax_wnd = [60] # potencia máxima gerador eólico

prob_cenario = [0.6,0.4] # probabilidade de ocorência dos cenários
p_wnd_real = pd.DataFrame([[50,10]], columns=scen, index=wnd) # produção eólica por cenários

# Previsão de produção da turbina eólica, considerando média ponderada dos cenários
p_wnd_forecast = np.dot(p_wnd_real.values, prob_cenario)

# Localização dos geradores térmicos
ger_loc = pd.DataFrame(np.array([[1,0], [1,0], [0,1]]),
                       columns=bus, index=ger)

# Localização dos geradores eólicos
wnd_loc = pd.DataFrame(np.array([[1,0]]),
                       columns=bus, index=wnd)

# Localização das cargas
load_loc = pd.DataFrame(np.array([[1,0], [0,1]]),
                        columns=bus, index=loads)

# Matriz de conexões entre barras
conex = pd.DataFrame(np.array([[0,1], [1,0]]),
                     columns=bus, index=bus)

# Matriz de reatâncias das linhas
x_line = pd.DataFrame(np.array([[0,0.13], [0.13,0]]),
                      columns=bus, index=bus)

# Matriz de limites térmicos das linhas
t_line = pd.DataFrame(np.array([[0,100], [100,0]]),
                      columns=bus, index=bus)

# Criar Modelo Concreto
model = ConcreteModel('mercado_conjunto_incerteza_renovavel')

#Criar Sets e seus parâmetros:

#Geradores Térmicos:
model.ger = Set(initialize=ger)
model.c_ener = Param(model.ger, initialize={ger[i]: C_ener[i] for i in range(Ng)})
model.c_up = Param(model.ger, initialize={ger[i]: C_up[i] for i in range(Ng)})
model.c_dw = Param(model.ger, initialize={ger[i]: C_dw[i] for i in range(Ng)})
model.pmax_ter = Param(model.ger, initialize={ger[i]: pmax_ter[i] for i in range(Ng)})

#Geradores Eólicos:
model.wnd = Set(initialize=wnd)
model.c_wind = Param(model.wnd, initialize={wnd[i]: C_wind[i] for i in range(Nw)})
model.pmax_wnd = Param(model.wnd, initialize={wnd[i]: pmax_wnd[i] for i in range(Nw)})
model.p_wnd_forecast = Param(model.wnd, initialize={wnd[i]:p_wnd_forecast[i] for i in range(Nw)})

#Cargas
model.loads = Set(initialize=loads)
model.dem = Param(model.loads, initialize={loads[i]: demanda_energia[i] for i in range(Nc)})
model.c_shed = Param(model.loads, initialize={loads[i]: C_shed[i] for i in range(Nc)})

#Cenários
model.scen = Set(initialize=scen)
model.prob = Param(model.scen, initialize={scen[i]: prob_cenario[i] for i in range(0,Ns)})
p_wnd_real_dict = {(wnd[i], scen[j]): p_wnd_real.loc[wnd[i], scen[j]] for i in range(Nw) for j in range(Ns)}# Criação do parâmetro p_wnd_real para representar a produção eólica por cenário e gerador eólico
model.p_wnd_real = Param(model.wnd, model.scen, initialize=p_wnd_real_dict)

#Barras e localizações:
model.bus = Set(initialize=bus)
model.ger_loc = Param(model.ger, model.bus, initialize=lambda model, g, b: ger_loc.loc[g, b], default=0, mutable=False)
model.wnd_loc = Param(model.wnd, model.bus, initialize=lambda model, w, b: wnd_loc.loc[w, b], default=0, mutable=False)
model.load_loc = Param(model.loads, model.bus, initialize=lambda model, l, b: load_loc.loc[l, b], default=0, mutable=False)
model.conex = Param(model.bus, model.bus, initialize=lambda model, b1, b2: conex.loc[b1, b2], default=0, mutable=False)
model.x_line = Param(model.bus, model.bus, initialize=lambda model, b1, b2: x_line.loc[b1, b2], default=0, mutable=False)
model.t_line = Param(model.bus, model.bus, initialize=lambda model, b1, b2: t_line.loc[b1, b2], default=0, mutable=False)

# Declarar Variáveis
model.P_ger = Var(model.ger, domain=NonNegativeReals) # Produção dos Geradores Térmicos
model.P_wnd = Var(model.wnd, domain=NonNegativeReals) # Produção dos Geradores Eólicos
model.R_up = Var(model.ger, domain=NonNegativeReals) # Reserva de Subida da Capacidade dos Produtores
model.R_dw = Var(model.ger, domain=NonNegativeReals) # Reserva de Descida da Capacidade dos Produtores
model.P_spill = Var(model.wnd,model.scen, domain=NonNegativeReals) # Descarte de Energia da Eólica (Spillage)
model.Reg_up = Var(model.ger,model.scen, domain=NonNegativeReals) # Ativação Reserva Subida Geradores Térmicos
model.Reg_dw = Var(model.ger,model.scen, domain=NonNegativeReals) # Ativação Reserva Descida Geradores Térmicos
model.L_shed = Var(model.loads,model.scen, domain=NonNegativeReals) # Corte de Carga
model.Theta_DA = Var(model.bus,domain=Reals) # Ângulos das Barras
model.Theta_RT = Var(model.bus,model.scen, domain=Reals) # Ângulos das Barras

#Restrições
def upper_bounds(model, ger): # restrição limite superior (ger)
    return model.P_ger[ger] + model.R_up[ger]<= model.pmax_ter[ger]
model.UpperBound = Constraint(model.ger, rule=upper_bounds)

def lower_bounds(model, ger): # restrição limite inferior (ger)
    return model.P_ger[ger] - model.R_dw[ger]>= 0
model.LowerBound = Constraint(model.ger, rule=lower_bounds)

def ger_wnd(model, wnd): # restrição geração eólica (wnd)
    return model.P_wnd[wnd] == model.p_wnd_forecast[wnd]
model.GerWnd = Constraint(model.wnd, rule=ger_wnd)

def day_ahead_balance(model, bus):  #Restrição do balanço de energia no mercado diário
    thermal_generation = sum(model.P_ger[g]*model.ger_loc[g,bus] for g in model.ger)
    wind_generation = sum(model.P_wnd[w]*model.wnd_loc[w,bus] for w in model.wnd)
    load = sum(model.dem[l]*model.load_loc[l,bus] for l in model.loads)
    line_flow = sum((model.Theta_DA[bus] - model.Theta_DA[j])/ model.x_line[bus,j] for j in model.bus if model.conex[bus,j]==1)
    return thermal_generation + wind_generation - load == line_flow
model.DA_Mkt = Constraint(model.bus, rule=day_ahead_balance)

def day_ahead_flow_limits(model, i, j): # restrição limite térmico da linha - tipo 1 (bus,bus)
    if i != j and model.conex[i,j] == 1:
        return (model.Theta_DA[i] - model.Theta_DA[j])/model.x_line[i,j] <= model.t_line[i,j]
    else:
        return Constraint.Skip
model.DA_Flow_Limits = Constraint(model.bus, model.bus, rule=day_ahead_flow_limits)

def operation_flow_limits(model, i, j, scen): # restrição limite térmico da linha - tipo 2 (bus,bus,scen)
     if i != j and model.conex[i,j] == 1:
        return (model.Theta_RT[i,scen] - model.Theta_RT[j,scen])/model.x_line[i,j] <= model.t_line[i,j]
     else:
        return Constraint.Skip
model.Operation_Flow_Limits = Constraint(model.bus, model.bus, model.scen, rule=operation_flow_limits)

def non_ant_up(model, ger, scen): # restrição non-anticipativity subida gerador (ger,scen)
    return model.Reg_up[ger,scen] <= model.R_up[ger]
model.NonAntUp = Constraint(model.ger, model.scen, rule=non_ant_up)

def non_ant_dw(model, ger, scen): # restrição non-anticipativity descida gerador (ger,scen)
    return model.Reg_dw[ger,scen] <= model.R_dw[ger]
model.NonAntDw = Constraint(model.ger, model.scen, rule=non_ant_dw)

def non_ant_shed(model, loads, scen): # restrição non-anticipativity corte carga (load,scen)
    return model.L_shed[loads,scen] <= model.dem[loads]
model.NonAntShed = Constraint(model.loads, model.scen, rule=non_ant_shed)

def non_ant_spillage(model, wnd, scen):  # restrição non-anticipativity spillage (wnd,scen)
    return model.P_spill[wnd,scen]<=model.p_wnd_real[wnd,scen]
model.NonAntSp = Constraint(model.wnd, model.scen, rule=non_ant_spillage)

def operation_balance(model, bus, scen): # restrição balanço de energia (bus,scen)
    thermal_reserve_ativation = sum((model.Reg_up[g,scen]-model.Reg_dw[g,scen])*model.ger_loc[g,bus] for g in model.ger)
    wind = sum((model.p_wnd_real[w,scen] - model.P_wnd[w] - model.P_spill[w,scen])*model.wnd_loc[w, bus] for w in model.wnd)
    load = sum(model.L_shed[l,scen]*model.load_loc[l,bus] for l in model.loads)
    line_flow = sum((model.Theta_RT[bus,scen] - model.Theta_DA[bus] + model.Theta_DA[j] - model.Theta_RT[j, scen])/model.x_line[bus,j] for j in model.bus if model.conex[bus, j] == 1)
    return thermal_reserve_ativation + wind + load == line_flow
model.Operation_Balance = Constraint(model.bus, model.scen, rule=operation_balance)

##Função Objetivo:
def fob(model):
    #Parte Determinística:
    custo_energia = sum(model.c_ener[g]*model.P_ger[g] for g in model.ger)
    custo_reserva_subida = sum(model.c_up[g]*model.R_up[g] for g in model.ger)
    custo_reserva_descida = sum(model.c_dw[g]*model.R_dw[g] for g in model.ger)
    custo_wnd = sum(model.c_wind[w]*model.P_wnd[w] for w in model.wnd)
    custo_deterministico = custo_energia + custo_reserva_subida + custo_reserva_descida + custo_wnd

    #Parte Probabilística:
    custo_probabilistico = 0
    for scen in model.scen:
        custo_reserva_ativada = sum((model.Reg_up[g, scen] - model.Reg_dw[g, scen]) * model.c_ener[g] for g in model.ger)
        custo_wnd_real = sum((model.p_wnd_real[w,scen] - model.P_wnd[w] - model.P_spill[w, scen])*model.c_wind[w] for w in model.wnd)
        custo_carga_cortada = sum(model.L_shed[l, scen] * model.c_shed[l] for l in model.loads)
        custo_probabilistico += model.prob[scen] * (custo_reserva_ativada + custo_wnd_real + custo_carga_cortada)
    custo_total = custo_deterministico + custo_probabilistico
    return custo_total
model.obj = Objective(rule=fob, sense=minimize)

solver = SolverFactory('glpk')
resultados = solver.solve(model, tee=False)

####Resultados

In [46]:
# Relatório dos resultados de otimização

print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo:', value(model.obj), '\n')

# prompt: printe os valores das variáveis desse problema de otimização

print("Produção dos Geradores Térmicos:")
for g in model.ger:
  print(f"  Gerador {g}: {model.P_ger[g].value} MWh")

print("\nProdução dos Geradores Eólicos:")
for w in model.wnd:
  print(f"  Gerador {w}: {model.P_wnd[w].value} MWh")

print("\nReserva de Subida da Capacidade dos Produtores:")
for g in model.ger:
  print(f"  Gerador {g}: {model.R_up[g].value} MW")

print("\nReserva de Descida da Capacidade dos Produtores:")
for g in model.ger:
  print(f"  Gerador {g}: {model.R_dw[g].value} MW")

print("\nDescarte de Energia da Eólica (Spillage):")
for w in model.wnd:
  for s in model.scen:
    print(f"  Gerador {w}, {s}: {model.P_spill[w,s].value} MW")

print("\nAtivação Reserva Subida Geradores Térmicos:")
for g in model.ger:
  for s in model.scen:
    print(f"  Gerador {g}, {s}: {model.Reg_up[g,s].value} MW")

print("\nAtivação Reserva Descida Geradores Térmicos:")
for g in model.ger:
  for s in model.scen:
    print(f"  Gerador {g}, {s}: {model.Reg_dw[g,s].value} MW")

print("\nCorte de Carga:")
for l in model.loads:
  for s in model.scen:
    print(f"  {l}, {s}: {model.L_shed[l,s].value} MWh")

print("\nÂngulos das Barras (Mercado Diário):")
for b in model.bus:
  print(f"  {b}: {model.Theta_DA[b].value} º")

print("\nÂngulos das Barras (Tempo Real):")
for b in model.bus:
  for s in model.scen:
    print(f"  {b}, {s}: {model.Theta_RT[b,s].value} º")

#Preço da produção:
print("\nPreço da Produção:")
for g in model.ger:
  print(f"  Gerador {g}: {model.P_ger[g].value*model.c_ener[g]} $")

print("\nPreço da Reserva Subida:")
for g in model.ger:
  print(f"  Gerador {g}: {model.R_up[g].value*model.c_up[g]} $") # Removed .value from model.c_up[g]

print("\nPreço da Reserva Descida:")
for g in model.ger:
  print(f"  Gerador {g}: {model.R_dw[g].value*model.c_dw[g]} $") # Removed .value from model.c_up[g]

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: 2644.0 

Produção dos Geradores Térmicos:
  Gerador Termico_1: 50.0 MWh
  Gerador Termico_2: 40.0 MWh
  Gerador Termico_3: 16.0 MWh

Produção dos Geradores Eólicos:
  Gerador Eolico_1: 34.0 MWh

Reserva de Subida da Capacidade dos Produtores:
  Gerador Termico_1: 0.0 MW
  Gerador Termico_2: 0.0 MW
  Gerador Termico_3: 24.0 MW

Reserva de Descida da Capacidade dos Produtores:
  Gerador Termico_1: 0.0 MW
  Gerador Termico_2: 0.0 MW
  Gerador Termico_3: 16.0 MW

Descarte de Energia da Eólica (Spillage):
  Gerador Eolico_1, Cenario_1: 0.0 MW
  Gerador Eolico_1, Cenario_2: 0.0 MW

Ativação Reserva Subida Geradores Térmicos:
  Gerador Termico_1, Cenario_1: 0.0 MW
  Gerador Termico_1, Cenario_2: 0.0 MW
  Gerador Termico_2, Cenario_1: 0.0 MW
  Gerador Termico_2, Cenario_2: 0.0 MW
  Gerador Termico_3, Cenario_1: 0.0 MW
  Gerador Termico_3, Cenario_2: 24.0 MW

Ativação Reserva Descida Geradores

###Duas Eólicas

####Modelo do Problema

In [47]:
# Dados Entrada

ger = ['Termico_1', 'Termico_2', 'Termico_3']
Ng = len(ger)

wnd = ['Eolico_1', 'Eolico_2']
Nw = len(wnd)

loads = ['Carga_1', 'Carga_2']
Nc = len(loads)

Ns = 10
scen = [f'Cenário{i+1}' for i in range(Ns)]
print(scen)

bus = ['Barra_1', 'Barra_2']
Nb = len(bus)

demanda_energia = [40,100] # por carga

C_ener = [10,30,35] # $/MWh por gerador térmico
C_up = [16,13,10]   # $/MWh por gerador térmico
C_dw = [15,12,9]    # $/MWh por gerador térmico
C_shed = [200,200]  # $/MWh por carga
C_wind = [0, 0]        # $/MWh por gerador eólico

pmax_ter = [50,110,100] # potencia máxima geradores térmicos

prob_cenario = [0.1]*Ns # probabilidade de cada cenário


p_wnd_real = pd.DataFrame([[5, 10, 15, 20, 25, 5, 10, 15, 20, 25],
                          [10, 15, 20, 25, 30, 35, 40, 45, 50, 30]],
                          columns=scen, index=wnd) # produção eólica por cenários


# Calculando previsão da produção eólica
p_wnd_forecast = np.dot(p_wnd_real.values, prob_cenario)

# Localização dos geradores térmicos
ger_loc = pd.DataFrame(np.array([[1,0], [1,0], [0,1]]),
                       columns=bus, index=ger)

# Localização dos geradores eólicos
wnd_loc = pd.DataFrame(np.array([[1,0], [0,1]]),
                       columns=bus, index=wnd)

# Localização das cargas
load_loc = pd.DataFrame(np.array([[1,0], [0,1]]),
                        columns=bus, index=loads)

# Matriz de conexões entre barras
conex = pd.DataFrame(np.array([[0,1], [1,0]]),
                     columns=bus, index=bus)

# Matriz de reatâncias das linhas
x_line = pd.DataFrame(np.array([[0,0.13], [0.13,0]]),
                      columns=bus, index=bus)

# Matriz de limites térmicos das linhas
t_line = pd.DataFrame(np.array([[0,100], [100,0]]),
                      columns=bus, index=bus)

# Criar Modelo Concreto
model = ConcreteModel('mercado_conjunto_incerteza_renovavel')

#Criar Sets e seus parâmetros:

#Geradores Térmicos:
model.ger = Set(initialize=ger)
model.c_ener = Param(model.ger, initialize={ger[i]: C_ener[i] for i in range(Ng)})
model.c_up = Param(model.ger, initialize={ger[i]: C_up[i] for i in range(Ng)})
model.c_dw = Param(model.ger, initialize={ger[i]: C_dw[i] for i in range(Ng)})
model.pmax_ter = Param(model.ger, initialize={ger[i]: pmax_ter[i] for i in range(Ng)})

#Geradores Eólicos:
model.wnd = Set(initialize=wnd)
model.c_wind = Param(model.wnd, initialize={wnd[i]: C_wind[i] for i in range(Nw)})
model.p_wnd_forecast = Param(model.wnd, initialize={wnd[i]:p_wnd_forecast[i] for i in range(Nw)})

#Cargas
model.loads = Set(initialize=loads)
model.dem = Param(model.loads, initialize={loads[i]: demanda_energia[i] for i in range(Nc)})
model.c_shed = Param(model.loads, initialize={loads[i]: C_shed[i] for i in range(Nc)})

#Cenários
model.scen = Set(initialize=scen)
model.prob = Param(model.scen, initialize={scen[i]: prob_cenario[i] for i in range(0,Ns)})
p_wnd_real_dict = {(wnd[i], scen[j]): p_wnd_real.loc[wnd[i], scen[j]] for i in range(Nw) for j in range(Ns)}# Criação do parâmetro p_wnd_real para representar a produção eólica por cenário e gerador eólico
model.p_wnd_real = Param(model.wnd, model.scen, initialize=p_wnd_real_dict)

#Barras e localizações:
model.bus = Set(initialize=bus)
model.ger_loc = Param(model.ger, model.bus, initialize=lambda model, g, b: ger_loc.loc[g, b], default=0, mutable=False)
model.wnd_loc = Param(model.wnd, model.bus, initialize=lambda model, w, b: wnd_loc.loc[w, b], default=0, mutable=False)
model.load_loc = Param(model.loads, model.bus, initialize=lambda model, l, b: load_loc.loc[l, b], default=0, mutable=False)
model.conex = Param(model.bus, model.bus, initialize=lambda model, b1, b2: conex.loc[b1, b2], default=0, mutable=False)
model.x_line = Param(model.bus, model.bus, initialize=lambda model, b1, b2: x_line.loc[b1, b2], default=0, mutable=False)
model.t_line = Param(model.bus, model.bus, initialize=lambda model, b1, b2: t_line.loc[b1, b2], default=0, mutable=False)

# Declarar Variáveis
model.P_ger = Var(model.ger, domain=NonNegativeReals) # Produção dos Geradores Térmicos
model.P_wnd = Var(model.wnd, domain=NonNegativeReals) # Produção dos Geradores Eólicos
model.R_up = Var(model.ger, domain=NonNegativeReals) # Reserva de Subida da Capacidade dos Produtores
model.R_dw = Var(model.ger, domain=NonNegativeReals) # Reserva de Descida da Capacidade dos Produtores
model.P_spill = Var(model.wnd,model.scen, domain=NonNegativeReals) # Descarte de Energia da Eólica (Spillage)
model.Reg_up = Var(model.ger,model.scen, domain=NonNegativeReals) # Ativação Reserva Subida Geradores Térmicos
model.Reg_dw = Var(model.ger,model.scen, domain=NonNegativeReals) # Ativação Reserva Descida Geradores Térmicos
model.L_shed = Var(model.loads,model.scen, domain=NonNegativeReals) # Corte de Carga
model.Theta_DA = Var(model.bus,domain=Reals) # Ângulos das Barras
model.Theta_RT = Var(model.bus,model.scen, domain=Reals) # Ângulos das Barras

def upper_bounds(model, ger): # restrição limite superior (ger)
    return model.P_ger[ger] + model.R_up[ger]<= model.pmax_ter[ger]
model.UpperBound = Constraint(model.ger, rule=upper_bounds)

def lower_bounds(model, ger): # restrição limite inferior (ger)
    return model.P_ger[ger] - model.R_dw[ger]>= 0
model.LowerBound = Constraint(model.ger, rule=lower_bounds)

def ger_wnd(model, wnd): # restrição geração eólica (wnd)
    return model.P_wnd[wnd] == model.p_wnd_forecast[wnd]
model.GerWnd = Constraint(model.wnd, rule=ger_wnd)

def day_ahead_balance(model, bus):  #Restrição do balanço de energia no mercado diário
    thermal_generation = sum(model.P_ger[g]*model.ger_loc[g,bus] for g in model.ger)
    wind_generation = sum(model.P_wnd[w]*model.wnd_loc[w,bus] for w in model.wnd)
    load = sum(model.dem[l]*model.load_loc[l,bus] for l in model.loads)
    line_flow = sum((model.Theta_DA[bus] - model.Theta_DA[j])/ model.x_line[bus,j] for j in model.bus if model.conex[bus,j]==1)
    return thermal_generation + wind_generation - load == line_flow
model.DA_Mkt = Constraint(model.bus, rule=day_ahead_balance)

def day_ahead_flow_limits(model, i, j): # restrição limite térmico da linha - tipo 1 (bus,bus)
    if i != j and model.conex[i,j] == 1:
        return (model.Theta_DA[i] - model.Theta_DA[j])/model.x_line[i,j] <= model.t_line[i,j]
    else:
        return Constraint.Skip
model.DA_Flow_Limits = Constraint(model.bus, model.bus, rule=day_ahead_flow_limits)

def operation_flow_limits(model, i, j, scen): # restrição limite térmico da linha - tipo 2 (bus,bus,scen)
     if i != j and model.conex[i,j] == 1:
        return (model.Theta_RT[i,scen] - model.Theta_RT[j,scen])/model.x_line[i,j] <= model.t_line[i,j]
     else:
        return Constraint.Skip
model.Operation_Flow_Limits = Constraint(model.bus, model.bus, model.scen, rule=operation_flow_limits)

def non_ant_up(model, ger, scen): # restrição non-anticipativity subida gerador (ger,scen)
    return model.Reg_up[ger,scen] <= model.R_up[ger]
model.NonAntUp = Constraint(model.ger, model.scen, rule=non_ant_up)

def non_ant_dw(model, ger, scen): # restrição non-anticipativity descida gerador (ger,scen)
    return model.Reg_dw[ger,scen] <= model.R_dw[ger]
model.NonAntDw = Constraint(model.ger, model.scen, rule=non_ant_dw)

def non_ant_shed(model, loads, scen): # restrição non-anticipativity corte carga (load,scen)
    return model.L_shed[loads,scen] <= model.dem[loads]
model.NonAntShed = Constraint(model.loads, model.scen, rule=non_ant_shed)

def non_ant_spillage(model, wnd, scen):  # restrição non-anticipativity spillage (wnd,scen)
    return model.P_spill[wnd,scen]<=model.p_wnd_real[wnd,scen]
model.NonAntSp = Constraint(model.wnd, model.scen, rule=non_ant_spillage)

def operation_balance(model, bus, scen): # restrição balanço de energia (bus,scen)
    thermal_reserve_ativation = sum((model.Reg_up[g,scen]-model.Reg_dw[g,scen])*model.ger_loc[g,bus] for g in model.ger)
    wind = sum((model.p_wnd_real[w,scen] - model.P_wnd[w] - model.P_spill[w,scen])*model.wnd_loc[w, bus] for w in model.wnd)
    load = sum(model.L_shed[l,scen]*model.load_loc[l,bus] for l in model.loads)
    line_flow = sum((model.Theta_RT[bus,scen] - model.Theta_DA[bus] + model.Theta_DA[j] - model.Theta_RT[j, scen])/model.x_line[bus,j] for j in model.bus if model.conex[bus, j] == 1)
    return thermal_reserve_ativation + wind + load == line_flow
model.Operation_Balance = Constraint(model.bus, model.scen, rule=operation_balance)

##Função Objetivo:
def fob(model):
    #Parte Determinística:
    custo_energia = sum(model.c_ener[g]*model.P_ger[g] for g in model.ger)
    custo_reserva_subida = sum(model.c_up[g]*model.R_up[g] for g in model.ger)
    custo_reserva_descida = sum(model.c_dw[g]*model.R_dw[g] for g in model.ger)
    custo_wnd = sum(model.c_wind[w]*model.P_wnd[w] for w in model.wnd)
    custo_deterministico = custo_energia + custo_reserva_subida + custo_reserva_descida + custo_wnd

    #Parte Probabilística:
    custo_probabilistico = 0
    for scen in model.scen:
        custo_reserva_ativada = sum((model.Reg_up[g, scen] - model.Reg_dw[g, scen]) * model.c_ener[g] for g in model.ger)
        custo_wnd_real = sum((model.p_wnd_real[w,scen] - model.P_wnd[w] - model.P_spill[w, scen])*model.c_wind[w] for w in model.wnd)
        custo_carga_cortada = sum(model.L_shed[l, scen] * model.c_shed[l] for l in model.loads)
        custo_probabilistico += model.prob[scen] * (custo_reserva_ativada + custo_wnd_real + custo_carga_cortada)
    custo_total = custo_deterministico + custo_probabilistico
    return custo_total
model.obj = Objective(rule=fob, sense=minimize)

solver = SolverFactory('glpk')
resultados = solver.solve(model, tee=False)


['Cenário1', 'Cenário2', 'Cenário3', 'Cenário4', 'Cenário5', 'Cenário6', 'Cenário7', 'Cenário8', 'Cenário9', 'Cenário10']


####Resultados

In [48]:
# Relatório dos resultados de otimização

print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo:', value(model.obj), '\n')

# prompt: printe os valores das variáveis desse problema de otimização

print("Produção dos Geradores Térmicos:")
for g in model.ger:
  print(f"  Gerador {g}: {model.P_ger[g].value} MWh")

print("\nProdução dos Geradores Eólicos:")
for w in model.wnd:
  print(f"  Gerador {w}: {model.P_wnd[w].value} MWh")

print("\nReserva de Subida da Capacidade dos Produtores:")
for g in model.ger:
  print(f"  Gerador {g}: {model.R_up[g].value} MW")

print("\nReserva de Descida da Capacidade dos Produtores:")
for g in model.ger:
  print(f"  Gerador {g}: {model.R_dw[g].value} MW")

print("\nDescarte de Energia da Eólica (Spillage):")
for w in model.wnd:
  for s in model.scen:
    print(f"  Gerador {w}, {s}: {model.P_spill[w,s].value} MW")

print("\nAtivação Reserva Subida Geradores Térmicos:")
for g in model.ger:
  for s in model.scen:
    print(f"  Gerador {g}, {s}: {model.Reg_up[g,s].value} MW")

print("\nAtivação Reserva Descida Geradores Térmicos:")
for g in model.ger:
  for s in model.scen:
    print(f"  Gerador {g}, {s}: {model.Reg_dw[g,s].value} MW")

print("\nCorte de Carga:")
for l in model.loads:
  for s in model.scen:
    print(f"  {l}, {s}: {model.L_shed[l,s].value} MWh")

print("\nÂngulos das Barras (Mercado Diário):")
for b in model.bus:
  print(f"  {b}: {model.Theta_DA[b].value} º")

print("\nÂngulos das Barras (Tempo Real):")
for b in model.bus:
  for s in model.scen:
    print(f"  {b}, {s}: {model.Theta_RT[b,s].value} º")

#Preço da produção:
print("\nPreço da Produção:")
for g in model.ger:
  print(f"  Gerador {g}: {model.P_ger[g].value*model.c_ener[g]} $")

print("\nPreço da Reserva Subida:")
for g in model.ger:
  print(f"  Gerador {g}: {model.R_up[g].value*model.c_up[g]} $") # Removed .value from model.c_up[g]

print("\nPreço da Reserva Descida:")
for g in model.ger:
  print(f"  Gerador {g}: {model.R_dw[g].value*model.c_dw[g]} $") # Removed .value from model.c_up[g]

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: 2360.0 

Produção dos Geradores Térmicos:
  Gerador Termico_1: 50.0 MWh
  Gerador Termico_2: 35.0 MWh
  Gerador Termico_3: 10.0 MWh

Produção dos Geradores Eólicos:
  Gerador Eolico_1: 15.0 MWh
  Gerador Eolico_2: 30.0 MWh

Reserva de Subida da Capacidade dos Produtores:
  Gerador Termico_1: 0.0 MW
  Gerador Termico_2: 5.0 MW
  Gerador Termico_3: 25.0 MW

Reserva de Descida da Capacidade dos Produtores:
  Gerador Termico_1: 0.0 MW
  Gerador Termico_2: 0.0 MW
  Gerador Termico_3: 10.0 MW

Descarte de Energia da Eólica (Spillage):
  Gerador Eolico_1, Cenário1: 0.0 MW
  Gerador Eolico_1, Cenário2: 0.0 MW
  Gerador Eolico_1, Cenário3: 0.0 MW
  Gerador Eolico_1, Cenário4: 0.0 MW
  Gerador Eolico_1, Cenário5: 0.0 MW
  Gerador Eolico_1, Cenário6: 0.0 MW
  Gerador Eolico_1, Cenário7: 0.0 MW
  Gerador Eolico_1, Cenário8: 5.0 MW
  Gerador Eolico_1, Cenário9: 15.0 MW
  Gerador Eolico_1, Cenário1

## Programação Robusta

##Chance Constrained - Big M

###Problema Inicial

####Modelo do Problema

In [49]:
num_variaveis = 2
w = [1, 2, 3, 4]
confianca = 0.75
X = [f'X {i+1}' for i in range(num_variaveis)]
Z = [f'Z {i+1}' for i in range(len(w))]
pi = [1 / len(w)] * len(w)
pi = pd.DataFrame(pi, index = w)

M = 3.5
model = ConcreteModel('Big M')

#Sets e Parâmetros
model.w = RangeSet(len(w))
model.pi = Param(model.w, initialize = pi)
model.confianca = Param(initialize = confianca)
model.M = Param(initialize = M)
model.i = RangeSet(num_variaveis)  # Para x_1 e x_2


# Variáveis
model.x = Var(model.i, domain=NonNegativeReals)
model.Z = Var(model.w, domain=Binary)

#Restrições
def restricao1(model, omega):
    return omega * model.x[1] + model.x[2] >= 7 - M * (1 - model.Z[omega])
model.restricao = Constraint(model.w, rule=restricao1)

def restricao2(model):
    return sum(model.Z[j] * model.pi[j] for j in model.w) >= model.confianca
model.restricao2 = Constraint(rule=restricao2)

# Função objetivo
model.obj = Objective(expr=sum(model.x[i] for i in model.i), sense=minimize)

# Resolvendo o modelo
solver = SolverFactory('glpk')
result = solver.solve(model)



####Resultados

In [50]:
# Exibindo os resultados
print("x_1:", model.x[1].value)
print("x_2:", model.x[2].value)
for k in model.w:
    print(f"Z_{k}:", model.Z[k].value)

x_1: 3.5
x_2: 0.0
Z_1: 0.0
Z_2: 1.0
Z_3: 1.0
Z_4: 1.0


**1)O que acontece se alterar o nível de confiança?**

As variáveis binárias de Z serão ativadas conforme o aumento da confiança, e serão desativadas com a diminuição.
Isso impacta nos valores das variáveis também. Com o aumento da confiança a variável x1 se aproxima de 7.

**2)O que acontece se o valor de M não for adequado?**
Com valores de M superiores a 3.5, X1 retornará 3.5 normalmente, porém com valores inferiores a 3.5 a variável X1 se aproxima de 7

###Demanda Incerta
Adicionando o Big M na restrição de balanço do problema da demanda incerta feita na aula de Mercado Conjunto

####Modelo do Problema

In [51]:
# Dados Entrada

ger = ['Produtor A', 'Produtor B']
Ng = len(ger)
pmax_ter = [100,100] # MW

loads = ['Consumidor 1']
Nc = len(loads)
demanda_energia = [130]

scen = ['Cenario_1', 'Cenario_2']
Ns = len(scen)
prob_cenario = [0.05,0.95] # probabilidade de ocorência dos cenários
dem_reserva_real = pd.DataFrame([[20,10]], columns = scen, index = loads) # Carga por cenários
dem_reserva_forecast = np.dot(dem_reserva_real, prob_cenario) # Forecast da Carga


C_ener = [10,30] # $/MWh
C_res = [0,25]  # $/MW
C_shed = [100]  # $/MWh por carga

# Criar Modelo Concreto
model = ConcreteModel()

#Geradores Térmicos:
model.ger = Set(initialize=ger)
model.c_ener = Param(model.ger, initialize={ger[i]: C_ener[i] for i in range(Ng)})
model.c_res = Param(model.ger, initialize={ger[i]: C_res[i] for i in range(Ng)})
model.pmax_ter = Param(model.ger, initialize={ger[i]: pmax_ter[i] for i in range(Ng)})

#Cargas:
model.loads = Set(initialize=loads)
model.dem = Param(model.loads, initialize={loads[i]: demanda_energia[i] for i in range(Nc)})
model.c_shed = Param(model.loads, initialize={loads[i]: C_shed[i] for i in range(Nc)})

#Cenários:
model.scen = Set(initialize=scen)
model.prob = Param(model.scen, initialize={scen[i]: prob_cenario[i] for i in range(Ns)})
dem_res_real = {(loads[i], scen[j]): dem_reserva_real.loc[loads[i], scen[j]] for i in range(Nc) for j in range(Ns)}
model.dem_res_real = Param(model.loads, model.scen, initialize=dem_res_real)
dem_res_forecast = Param(model.loads, initialize={loads[i]: dem_reserva_forecast[i] for i in range(Nc)})

# Declarar Variáveis
model.P_ger = Var(model.ger, domain=NonNegativeReals)
model.R = Var(model.ger, domain=NonNegativeReals)
model.r = Var(model.ger, model.scen, domain=NonNegativeReals)
model.L_shed = Var(model.loads, model.scen, domain = NonNegativeReals)
model.z = Var(model.scen, domain=Binary) #Variável Binária do BIG M

#Restrições
def day_ahead_balance(model): #Restrição do balanço de energia no mercado diário
    thermal_generation =  sum(model.P_ger[g] for g in model.ger)
    demand = sum(model.dem[l] for l in model.loads)
    return thermal_generation == demand
model.day_ahead_balance = Constraint(rule=day_ahead_balance)

def upper_bounds(model, ger): # restrição 3: limite de geração
    return model.P_ger[ger] + model.R[ger] <= model.pmax_ter[ger]
model.UpperBound = Constraint(model.ger, rule=upper_bounds)

def non_ant_res(model, ger, cen):
    return model.r[ger,cen] <= model.R[ger]
model.NonAntRes = Constraint(model.ger, model.scen, rule=non_ant_res)

def non_ant_shed(model, carga, scen): # restrição non-anticipativity corte carga (load,scen)
    return model.L_shed[carga,scen] <= model.dem_res_real[carga,scen]
model.NonAntShed = Constraint(model.loads, model.scen, rule=non_ant_shed)

#Restrições do Big M:
confianca = 0.95
M = 1000

def operation_balance_big_M_1(model, carga, cen): #Restrição do balanço de ativação de reserva
    thermal_reserve_ativation = sum(model.r[g,cen] for g in model.ger)
    shed_load = sum(model.L_shed[l,cen] for l in model.loads)
    return thermal_reserve_ativation + shed_load - model.dem_res_real[carga,cen] >= -M*(1-model.z[cen])
model.operation_balance_big_M_1 = Constraint(model.loads, model.scen, rule=operation_balance_big_M_1)

def operation_balance_big_M_2(model, carga, cen): #Restrição do balanço de ativação de reserva
    thermal_reserve_ativation = sum(model.r[g,cen] for g in model.ger)
    shed_load = sum(model.L_shed[l,cen] for l in model.loads)
    return thermal_reserve_ativation + shed_load - model.dem_res_real[carga,cen] <= M*(1-model.z[cen])
model.operation_balance_big_M_2 = Constraint(model.loads, model.scen, rule=operation_balance_big_M_2)

def restricao_adicional_big_M(model):
    return sum(model.z[cen] * model.prob[cen] for cen in model.scen) >= confianca
model.restricao_adicional = Constraint(rule=restricao_adicional_big_M)

# Declarar Função Objetivo
def FOB(model):
    #Parte Determinística
    custo_energia = sum(model.c_ener[g]*model.P_ger[g] for g in model.ger)
    custo_reserva = sum(model.c_res[g]*model.R[g] for g in model.ger)
    FOB_det = custo_energia + custo_reserva

    #Parte Probabilística
    FOB_prob = 0
    for cen in model.scen:
        custo_reserva_ativada = sum(model.c_ener[g]*model.r[g,cen] for g in model.ger)
        custo_deslastre = sum(model.c_shed[l]*model.L_shed[l,cen] for l in model.loads)
        FOB_prob += model.prob[cen] * (custo_reserva_ativada + custo_deslastre)
    return FOB_det + FOB_prob

model.objective = Objective(rule=FOB, sense=minimize)

solver = SolverFactory('glpk')
# solver = SolverFactory('ipopt',executable='/content/ipopt')

resultados = solver.solve(model, tee=False)

####Resultados

In [52]:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo:', value(model.objective), '\n')

#Print das Variáveis
for i in model.ger:
    print(f'Produção do {i}:' ,value(model.P_ger[i]), '\n')
    print(f'Reserva do {i}:' ,value(model.R[i]), '\n')


for j in model.scen:
    for i in model.loads:
        print(f'Deslastre do {i} no {j}:', value(model.L_shed[i,j]), '\n')
    for i in model.ger:
        print(f'Reserva ativada do {i} no {j}:' ,value(model.r[i,j]), '\n')

#Variáveis de decisão Z:
for i in model.scen:
    print(f'Z_{i}:' ,value(model.z[i]), '\n')

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: 2195.0 

Produção do Produtor A: 90.0 

Reserva do Produtor A: 10.0 

Produção do Produtor B: 40.0 

Reserva do Produtor B: 0.0 

Deslastre do Consumidor 1 no Cenario_1: 0.0 

Reserva ativada do Produtor A no Cenario_1: 0.0 

Reserva ativada do Produtor B no Cenario_1: 0.0 

Deslastre do Consumidor 1 no Cenario_2: 0.0 

Reserva ativada do Produtor A no Cenario_2: 10.0 

Reserva ativada do Produtor B no Cenario_2: 0.0 

Z_Cenario_1: 0.0 

Z_Cenario_2: 1.0 



###Uma Eólica
Aplicando Big M para relaxar restrição de balanço do problema da Eólica feito na aula de Mercado Conjunto

####Modelo do Problema

In [53]:
# Dados Entrada

ger = ['Termico_1', 'Termico_2', 'Termico_3']
Ng = len(ger)

wnd = ['Eolico_1']
Nw = len(wnd)

loads = ['Carga_1', 'Carga_2']
Nc = len(loads)

scen = ['Cenario_1', 'Cenario_2']
Ns = len(scen)

bus = ['Barra_1', 'Barra_2']
Nb = len(bus)

demanda_energia = [40,100] # por carga

C_ener = [10,30,35] # $/MWh por gerador térmico
C_up = [16,13,10]   # $/MWh por gerador térmico
C_dw = [15,12,9]    # $/MWh por gerador térmico
C_shed = [200,200]  # $/MWh por carga
C_wind = [0]        # $/MWh por gerador eólico

pmax_ter = [50,110,100] # potencia máxima geradores térmicos
pmax_wnd = [60] # potencia máxima gerador eólico

prob_cenario = [0.6,0.4] # probabilidade de ocorência dos cenários
p_wnd_real = pd.DataFrame([[50,10]], columns=scen, index=wnd) # produção eólica por cenários

# Previsão de produção da turbina eólica, considerando média ponderada dos cenários
p_wnd_forecast = np.dot(p_wnd_real.values, prob_cenario)

# Localização dos geradores térmicos
ger_loc = pd.DataFrame(np.array([[1,0], [1,0], [0,1]]),
                       columns=bus, index=ger)

# Localização dos geradores eólicos
wnd_loc = pd.DataFrame(np.array([[1,0]]),
                       columns=bus, index=wnd)

# Localização das cargas
load_loc = pd.DataFrame(np.array([[1,0], [0,1]]),
                        columns=bus, index=loads)

# Matriz de conexões entre barras
conex = pd.DataFrame(np.array([[0,1], [1,0]]),
                     columns=bus, index=bus)

# Matriz de reatâncias das linhas
x_line = pd.DataFrame(np.array([[0,0.13], [0.13,0]]),
                      columns=bus, index=bus)

# Matriz de limites térmicos das linhas
t_line = pd.DataFrame(np.array([[0,100], [100,0]]),
                      columns=bus, index=bus)

# Criar Modelo Concreto
model = ConcreteModel('mercado_conjunto_incerteza_renovavel')

#Criar Sets e seus parâmetros:

#Geradores Térmicos:
model.ger = Set(initialize=ger)
model.c_ener = Param(model.ger, initialize={ger[i]: C_ener[i] for i in range(Ng)})
model.c_up = Param(model.ger, initialize={ger[i]: C_up[i] for i in range(Ng)})
model.c_dw = Param(model.ger, initialize={ger[i]: C_dw[i] for i in range(Ng)})
model.pmax_ter = Param(model.ger, initialize={ger[i]: pmax_ter[i] for i in range(Ng)})

#Geradores Eólicos:
model.wnd = Set(initialize=wnd)
model.c_wind = Param(model.wnd, initialize={wnd[i]: C_wind[i] for i in range(Nw)})
model.pmax_wnd = Param(model.wnd, initialize={wnd[i]: pmax_wnd[i] for i in range(Nw)})
model.p_wnd_forecast = Param(model.wnd, initialize={wnd[i]:p_wnd_forecast[i] for i in range(Nw)})

#Cargas
model.loads = Set(initialize=loads)
model.dem = Param(model.loads, initialize={loads[i]: demanda_energia[i] for i in range(Nc)})
model.c_shed = Param(model.loads, initialize={loads[i]: C_shed[i] for i in range(Nc)})

#Cenários
model.scen = Set(initialize=scen)
model.prob = Param(model.scen, initialize={scen[i]: prob_cenario[i] for i in range(0,Ns)})
p_wnd_real_dict = {(wnd[i], scen[j]): p_wnd_real.loc[wnd[i], scen[j]] for i in range(Nw) for j in range(Ns)}# Criação do parâmetro p_wnd_real para representar a produção eólica por cenário e gerador eólico
model.p_wnd_real = Param(model.wnd, model.scen, initialize=p_wnd_real_dict)

#Barras e localizações:
model.bus = Set(initialize=bus)
model.ger_loc = Param(model.ger, model.bus, initialize=lambda model, g, b: ger_loc.loc[g, b], default=0, mutable=False)
model.wnd_loc = Param(model.wnd, model.bus, initialize=lambda model, w, b: wnd_loc.loc[w, b], default=0, mutable=False)
model.load_loc = Param(model.loads, model.bus, initialize=lambda model, l, b: load_loc.loc[l, b], default=0, mutable=False)
model.conex = Param(model.bus, model.bus, initialize=lambda model, b1, b2: conex.loc[b1, b2], default=0, mutable=False)
model.x_line = Param(model.bus, model.bus, initialize=lambda model, b1, b2: x_line.loc[b1, b2], default=0, mutable=False)
model.t_line = Param(model.bus, model.bus, initialize=lambda model, b1, b2: t_line.loc[b1, b2], default=0, mutable=False)

# Declarar Variáveis
model.P_ger = Var(model.ger, domain=NonNegativeReals) # Produção dos Geradores Térmicos
model.P_wnd = Var(model.wnd, domain=NonNegativeReals) # Produção dos Geradores Eólicos
model.R_up = Var(model.ger, domain=NonNegativeReals) # Reserva de Subida da Capacidade dos Produtores
model.R_dw = Var(model.ger, domain=NonNegativeReals) # Reserva de Descida da Capacidade dos Produtores
model.P_spill = Var(model.wnd,model.scen, domain=NonNegativeReals) # Descarte de Energia da Eólica (Spillage)
model.Reg_up = Var(model.ger,model.scen, domain=NonNegativeReals) # Ativação Reserva Subida Geradores Térmicos
model.Reg_dw = Var(model.ger,model.scen, domain=NonNegativeReals) # Ativação Reserva Descida Geradores Térmicos
model.L_shed = Var(model.loads,model.scen, domain=NonNegativeReals) # Corte de Carga
model.Theta_DA = Var(model.bus,domain=Reals) # Ângulos das Barras
model.Theta_RT = Var(model.bus,model.scen, domain=Reals) # Ângulos das Barras

def upper_bounds(model, ger): # restrição limite superior (ger)
    return model.P_ger[ger] + model.R_up[ger]<= model.pmax_ter[ger]
model.UpperBound = Constraint(model.ger, rule=upper_bounds)

def lower_bounds(model, ger): # restrição limite inferior (ger)
    return model.P_ger[ger] - model.R_dw[ger]>= 0
model.LowerBound = Constraint(model.ger, rule=lower_bounds)

def ger_wnd(model, wnd): # restrição geração eólica (wnd)
    return model.P_wnd[wnd] == model.p_wnd_forecast[wnd]
model.GerWnd = Constraint(model.wnd, rule=ger_wnd)

def day_ahead_balance(model, bus):  #Restrição do balanço de energia no mercado diário
    thermal_generation = sum(model.P_ger[g]*model.ger_loc[g,bus] for g in model.ger)
    wind_generation = sum(model.P_wnd[w]*model.wnd_loc[w,bus] for w in model.wnd)
    load = sum(model.dem[l]*model.load_loc[l,bus] for l in model.loads)
    line_flow = sum((model.Theta_DA[bus] - model.Theta_DA[j])/ model.x_line[bus,j] for j in model.bus if model.conex[bus,j]==1)
    return thermal_generation + wind_generation - load == line_flow
model.DA_Mkt = Constraint(model.bus, rule=day_ahead_balance)

def day_ahead_flow_limits(model, i, j): # restrição limite térmico da linha - tipo 1 (bus,bus)
    if i != j and model.conex[i,j] == 1:
        return (model.Theta_DA[i] - model.Theta_DA[j])/model.x_line[i,j] <= model.t_line[i,j]
    else:
        return Constraint.Skip
model.DA_Flow_Limits = Constraint(model.bus, model.bus, rule=day_ahead_flow_limits)

def operation_flow_limits(model, i, j, scen): # restrição limite térmico da linha - tipo 2 (bus,bus,scen)
     if i != j and model.conex[i,j] == 1:
        return (model.Theta_RT[i,scen] - model.Theta_RT[j,scen])/model.x_line[i,j] <= model.t_line[i,j]
     else:
        return Constraint.Skip
model.Operation_Flow_Limits = Constraint(model.bus, model.bus, model.scen, rule=operation_flow_limits)

def non_ant_up(model, ger, scen): # restrição non-anticipativity subida gerador (ger,scen)
    return model.Reg_up[ger,scen] <= model.R_up[ger]
model.NonAntUp = Constraint(model.ger, model.scen, rule=non_ant_up)

def non_ant_dw(model, ger, scen): # restrição non-anticipativity descida gerador (ger,scen)
    return model.Reg_dw[ger,scen] <= model.R_dw[ger]
model.NonAntDw = Constraint(model.ger, model.scen, rule=non_ant_dw)

def non_ant_shed(model, loads, scen): # restrição non-anticipativity corte carga (load,scen)
    return model.L_shed[loads,scen] <= model.dem[loads]
model.NonAntShed = Constraint(model.loads, model.scen, rule=non_ant_shed)

def non_ant_spillage(model, wnd, scen):  # restrição non-anticipativity spillage (wnd,scen)
    return model.P_spill[wnd,scen]<=model.p_wnd_real[wnd,scen]
model.NonAntSp = Constraint(model.wnd, model.scen, rule=non_ant_spillage)

#BIG M para restrição do balanço de reserva

model.z = Var(model.scen, domain=Binary) #Variável Binária do BIG M:
confianca = 0.6
M = 1000

def operation_balance_big_M_1(model, bus, scen): #Restrição do balanço de ativação de reserva
    thermal_reserve_ativation = sum((model.Reg_up[g,scen]-model.Reg_dw[g,scen])*model.ger_loc[g,bus] for g in model.ger)
    wind = sum((model.p_wnd_real[w,scen] - model.P_wnd[w] - model.P_spill[w,scen])*model.wnd_loc[w, bus] for w in model.wnd)
    load = sum(model.L_shed[l,scen]*model.load_loc[l,bus] for l in model.loads)
    line_flow = sum((model.Theta_RT[bus,scen] - model.Theta_DA[bus] + model.Theta_DA[j] - model.Theta_RT[j, scen])/model.x_line[bus,j] for j in model.bus if model.conex[bus, j] == 1)
    return thermal_reserve_ativation + wind + load - line_flow >= -M*(1-model.z[scen])
model.operation_balance_big_M_1 = Constraint(model.bus, model.scen, rule=operation_balance_big_M_1)

def operation_balance_big_M_2(model, bus, scen): #Restrição do balanço de ativação de reserva
    thermal_reserve_ativation = sum((model.Reg_up[g,scen]-model.Reg_dw[g,scen])*model.ger_loc[g,bus] for g in model.ger)
    wind = sum((model.p_wnd_real[w,scen] - model.P_wnd[w] - model.P_spill[w,scen])*model.wnd_loc[w, bus] for w in model.wnd)
    load = sum(model.L_shed[l,scen]*model.load_loc[l,bus] for l in model.loads)
    line_flow = sum((model.Theta_RT[bus,scen] - model.Theta_DA[bus] + model.Theta_DA[j] - model.Theta_RT[j, scen])/model.x_line[bus,j] for j in model.bus if model.conex[bus, j] == 1)
    return thermal_reserve_ativation + wind + load - line_flow <= M*(1-model.z[scen])
model.operation_balance_big_M_2 = Constraint(model.bus, model.scen, rule=operation_balance_big_M_2)

def restricao_adicional_big_M(model):
    return sum(model.z[cen] * model.prob[cen] for cen in model.scen) >= confianca
model.restricao_adicional = Constraint(rule=restricao_adicional_big_M)

##Função Objetivo:
def fob(model):
    #Parte Determinística:
    custo_energia = sum(model.c_ener[g]*model.P_ger[g] for g in model.ger)
    custo_reserva_subida = sum(model.c_up[g]*model.R_up[g] for g in model.ger)
    custo_reserva_descida = sum(model.c_dw[g]*model.R_dw[g] for g in model.ger)
    custo_wnd = sum(model.c_wind[w]*model.P_wnd[w] for w in model.wnd)
    custo_deterministico = custo_energia + custo_reserva_subida + custo_reserva_descida + custo_wnd

    #Parte Probabilística:
    custo_probabilistico = 0
    for scen in model.scen:
        custo_reserva_ativada = sum((model.Reg_up[g, scen] - model.Reg_dw[g, scen]) * model.c_ener[g] for g in model.ger)
        custo_wnd_real = sum((model.p_wnd_real[w,scen] - model.P_wnd[w] - model.P_spill[w, scen])*model.c_wind[w] for w in model.wnd)
        custo_carga_cortada = sum(model.L_shed[l, scen] * model.c_shed[l] for l in model.loads)
        custo_probabilistico += model.prob[scen] * (custo_reserva_ativada + custo_wnd_real + custo_carga_cortada)
    custo_total = custo_deterministico + custo_probabilistico
    return custo_total
model.obj = Objective(rule=fob, sense=minimize)

solver = SolverFactory('glpk')
resultados = solver.solve(model, tee=False)


####Resultados

In [54]:
# Relatório dos resultados de otimização

print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo:', value(model.obj), '\n')

# prompt: printe os valores das variáveis desse problema de otimização

print("Produção dos Geradores Térmicos:")
for g in model.ger:
  print(f"  Gerador {g}: {model.P_ger[g].value} MWh")

print("\nProdução dos Geradores Eólicos:")
for w in model.wnd:
  print(f"  Gerador {w}: {model.P_wnd[w].value} MWh")

print("\nReserva de Subida da Capacidade dos Produtores:")
for g in model.ger:
  print(f"  Gerador {g}: {model.R_up[g].value} MW")

print("\nReserva de Descida da Capacidade dos Produtores:")
for g in model.ger:
  print(f"  Gerador {g}: {model.R_dw[g].value} MW")

print("\nDescarte de Energia da Eólica (Spillage):")
for w in model.wnd:
  for s in model.scen:
    print(f"  Gerador {w}, {s}: {model.P_spill[w,s].value} MW")

print("\nAtivação Reserva Subida Geradores Térmicos:")
for g in model.ger:
  for s in model.scen:
    print(f"  Gerador {g}, {s}: {model.Reg_up[g,s].value} MW")

print("\nAtivação Reserva Descida Geradores Térmicos:")
for g in model.ger:
  for s in model.scen:
    print(f"  Gerador {g}, {s}: {model.Reg_dw[g,s].value} MW")

print("\nCorte de Carga:")
for l in model.loads:
  for s in model.scen:
    print(f"  {l}, {s}: {model.L_shed[l,s].value} MWh")

print("\nÂngulos das Barras (Mercado Diário):")
for b in model.bus:
  print(f"  {b}: {model.Theta_DA[b].value} º")

print("\nÂngulos das Barras (Tempo Real):")
for b in model.bus:
  for s in model.scen:
    print(f"  {b}, {s}: {model.Theta_RT[b,s].value} º")

#Preço da produção:
print("\nPreço da Produção:")
for g in model.ger:
  print(f"  Gerador {g}: {model.P_ger[g].value*model.c_ener[g]} $")

print("\nPreço da Reserva Subida:")
for g in model.ger:
  print(f"  Gerador {g}: {model.R_up[g].value*model.c_up[g]} $") # Removed .value from model.c_up[g]

print("\nPreço da Reserva Descida:")
for g in model.ger:
  print(f"  Gerador {g}: {model.R_dw[g].value*model.c_dw[g]} $") # Removed .value from model.c_up[g]

#Variáveis de decisão Z:
for i in model.scen:
    print(f'Z_{i}:' ,value(model.z[i]), '\n')

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: 1844.0 

Produção dos Geradores Térmicos:
  Gerador Termico_1: 50.0 MWh
  Gerador Termico_2: 0.0 MWh
  Gerador Termico_3: 56.0 MWh

Produção dos Geradores Eólicos:
  Gerador Eolico_1: 34.0 MWh

Reserva de Subida da Capacidade dos Produtores:
  Gerador Termico_1: 0.0 MW
  Gerador Termico_2: 0.0 MW
  Gerador Termico_3: 0.0 MW

Reserva de Descida da Capacidade dos Produtores:
  Gerador Termico_1: 0.0 MW
  Gerador Termico_2: 0.0 MW
  Gerador Termico_3: 56.0 MW

Descarte de Energia da Eólica (Spillage):
  Gerador Eolico_1, Cenario_1: 0.0 MW
  Gerador Eolico_1, Cenario_2: 0.0 MW

Ativação Reserva Subida Geradores Térmicos:
  Gerador Termico_1, Cenario_1: 0.0 MW
  Gerador Termico_1, Cenario_2: 0.0 MW
  Gerador Termico_2, Cenario_1: 0.0 MW
  Gerador Termico_2, Cenario_2: 0.0 MW
  Gerador Termico_3, Cenario_1: 0.0 MW
  Gerador Termico_3, Cenario_2: 0.0 MW

Ativação Reserva Descida Geradores Té

##Chance Constrained - McCormick

###Problema Inicial

####Modelo do Problema

In [55]:
# Definições de parâmetros
num_variaveis = 2
w = [1, 2, 3, 4]
confianca = 0.75
X = [f'X {i+1}' for i in range(num_variaveis)]
Z = [f'Z {i+1}' for i in range(len(w))]
pi = [1 / len(w)] * len(w)
pi = pd.DataFrame(pi, index = w)

# Definindo os parâmetros cmáximos e mínimos
x_min = [0, 0]  # x_min para x_1 e x_2
x_max = [10, 10]  # x_max para x_1 e x_2
z_min = [0, 0, 0, 0]  # z_min para Z_1, Z_2, Z_3, Z_4
z_max = [1, 1, 1, 1]  # z_max para Z_1, Z_2, Z_3, Z_4

# Criando o modelo concreto
model = ConcreteModel('McCormick')

# Sets e Parâmetros
model.w = RangeSet(len(w))
model.i = RangeSet(len(x_min))  # Definindo para as variáveis x
model.confianca = Param(initialize=confianca)

# Inicializando parâmetros de pi (probabilidades) como dicionário
pi_values = {i+1: 1 / len(w) for i in range(len(w))}
model.pi = Param(model.w, initialize=pi_values)

# Inicializando parâmetros de x_min, x_max, z_min e z_max
model.x_min = Param(model.i, initialize={i+1: x_min[i] for i in range(len(x_min))})
model.x_max = Param(model.i, initialize={i+1: x_max[i] for i in range(len(x_max))})
model.z_min = Param(model.w, initialize={i+1: z_min[i] for i in range(len(z_min))})
model.z_max = Param(model.w, initialize={i+1: z_max[i] for i in range(len(z_max))})

# Definindo as variáveis de decisão
model.X = Var(model.i, domain=NonNegativeReals)
model.Z = Var(model.w, domain=Binary)
model.k = Var(model.w, model.i, domain=NonNegativeReals)

# Função objetivo (minimizar a soma das variáveis X)
model.obj = Objective(expr=sum(model.X[i] for i in model.i), sense=minimize)

# Restrições de McCormick (aquelas que ligam X e Z com a variável k)
model.Cor = ConstraintList()
for i in model.i:
    for w in model.w:
        model.Cor.add(expr=model.x_min[i]*model.Z[w] + model.X[i]*model.z_min[w] - model.x_min[i]*model.z_min[w] <= model.k[w,i])
        model.Cor.add(expr=model.x_max[i]*model.Z[w] + model.X[i]*model.z_max[w] - model.x_max[i]*model.z_max[w] <= model.k[w,i])
        model.Cor.add(expr=model.x_max[i]*model.Z[w] + model.X[i]*model.z_min[w] - model.x_max[i]*model.z_min[w] >= model.k[w,i])
        model.Cor.add(expr=model.x_min[i]*model.Z[w] + model.X[i]*model.z_max[w] - model.x_min[i]*model.z_max[w] >= model.k[w,i])

# Restrições de probabilidade (relacionadas à confiança)
model.pi_constraints = ConstraintList()
for w in model.w:
    model.pi_constraints.add(expr=sum(model.Z[w]*model.pi[w] for w in model.w) >= model.confianca)

# Restrições adicionais de k e Z
model.kZ_constraints = ConstraintList()
for w in model.w:
    model.kZ_constraints.add(expr=w*model.k[w,1] + model.k[w,2] - 7*model.Z[w] >= 0)

# Resolver o modelo
opt = SolverFactory('glpk')
opt.solve(model, tee=False)

{'Problem': [{'Name': 'unknown', 'Lower bound': 3.5, 'Upper bound': 3.5, 'Number of objectives': 1, 'Number of constraints': 40, 'Number of variables': 14, 'Number of nonzeros': 92, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '3', 'Number of created subproblems': '3'}}, 'Error rc': 0, 'Time': 0.007641792297363281}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

####Resultados

In [56]:
# Exibindo resultados
for i in model.i:
    print(f"X {i+1}: {model.X[i].value}")
for w in model.w:
    print(f"Z {w}: {model.Z[w].value}")

X 2: 3.5
X 3: 0.0
Z 1: 0.0
Z 2: 1.0
Z 3: 1.0
Z 4: 1.0


###Demanda Incerta

####Modelo do Problema

In [57]:
# Dados Entrada

ger = ['Produtor A', 'Produtor B']
Ng = len(ger)
pmax_ter = [100,100] # MW

loads = ['Consumidor 1']
Nc = len(loads)
demanda_energia = [130]

scen = ['Cenario_1', 'Cenario_2']
Ns = len(scen)
prob_cenario = [0.05,0.95] # probabilidade de ocorência dos cenários
dem_reserva_real = pd.DataFrame([[20,10]], columns = scen, index = loads) # Carga por cenários
dem_reserva_forecast = np.dot(dem_reserva_real, prob_cenario) # Forecast da Carga


C_ener = [10,30] # $/MWh
C_res = [0,25]  # $/MW
C_shed = [100]  # $/MWh por carga

# Criar Modelo Concreto
model = ConcreteModel()

#Geradores Térmicos:
model.ger = Set(initialize=ger)
model.c_ener = Param(model.ger, initialize={ger[i]: C_ener[i] for i in range(Ng)})
model.c_res = Param(model.ger, initialize={ger[i]: C_res[i] for i in range(Ng)})
model.pmax_ter = Param(model.ger, initialize={ger[i]: pmax_ter[i] for i in range(Ng)})

#Cargas:
model.loads = Set(initialize=loads)
model.dem = Param(model.loads, initialize={loads[i]: demanda_energia[i] for i in range(Nc)})
model.c_shed = Param(model.loads, initialize={loads[i]: C_shed[i] for i in range(Nc)})

#Cenários:
model.scen = Set(initialize=scen)
model.prob = Param(model.scen, initialize={scen[i]: prob_cenario[i] for i in range(Ns)})
dem_res_real = {(loads[i], scen[j]): dem_reserva_real.loc[loads[i], scen[j]] for i in range(Nc) for j in range(Ns)}
model.dem_res_real = Param(model.loads, model.scen, initialize=dem_res_real)
dem_res_forecast = Param(model.loads, initialize={loads[i]: dem_reserva_forecast[i] for i in range(Nc)})

# Declarar Variáveis
model.P_ger = Var(model.ger, domain=NonNegativeReals)
model.R = Var(model.ger, domain=NonNegativeReals)
model.r = Var(model.ger, model.scen, domain=NonNegativeReals)
model.L_shed = Var(model.loads, model.scen, domain = NonNegativeReals)

#Restrições
def day_ahead_balance(model): #Restrição do balanço de energia no mercado diário
    thermal_generation =  sum(model.P_ger[g] for g in model.ger)
    demand = sum(model.dem[l] for l in model.loads)
    return thermal_generation == demand
model.day_ahead_balance = Constraint(rule=day_ahead_balance)

def upper_bounds(model, ger): # restrição 3: limite de geração
    return model.P_ger[ger] + model.R[ger] <= model.pmax_ter[ger]
model.UpperBound = Constraint(model.ger, rule=upper_bounds)

def non_ant_res(model, ger, cen):
    return model.r[ger,cen] <= model.R[ger]
model.NonAntRes = Constraint(model.ger, model.scen, rule=non_ant_res)

##MCCormick:
confianca = 0.95
#Variáveis: r e L_shed
model.z = Var(model.scen, domain=Binary)
model.k_r = Var(model.ger, model.scen, domain = Reals)
model.k_Lshed = Var(model.loads, model.scen, domain = Reals)

r_min = [0]*len(ger)
model.r_min = Param(model.ger, initialize={ger[i]: r_min[i] for i in range(Ng)})

L_shed_min = [0]*len(loads)
model.L_shed_min = Param(model.loads, initialize={loads[i]: L_shed_min[i] for i in range(Nc)})



#Restrições mccormick para r:
def mccormick_1_r(model, variavel, cenario):
      return model.k_r[variavel, cenario] >= model.r_min[variavel] * model.z[cenario]

def mccormick_2_r(model, variavel, cenario):
      return model.k_r[variavel, cenario] >= model.R[variavel] * model.z[cenario] + model.r[variavel, cenario] - model.R[variavel]

def mccormick_3_r(model, variavel, cenario):
      return model.k_r[variavel, cenario] <= model.R[variavel] * model.z[cenario]

def mccormick_4_r(model, variavel, cenario):
      return model.k_r[variavel, cenario] <= model.r[variavel, cenario] + model.r_min[variavel] * model.z[cenario] - model.r_min[variavel]

model.mccormick_1_r = Constraint(model.ger, model.scen, rule=mccormick_1_r)
model.mccormick_2_r = Constraint(model.ger, model.scen, rule=mccormick_2_r)
model.mccormick_3_r = Constraint(model.ger, model.scen, rule=mccormick_3_r)
model.mccormick_4_r = Constraint(model.ger, model.scen, rule=mccormick_4_r)

#Restrições mccormick para L_shed:
def mccormick_1_Lshed(model, variavel, cenario):
      return model.k_Lshed[variavel, cenario] >= model.L_shed_min[variavel] * model.z[cenario]

def mccormick_2_Lshed(model, variavel, cenario):
      return model.k_Lshed[variavel, cenario] >= model.dem_res_real[variavel, cenario] * model.z[cenario] + model.L_shed[variavel, cenario] - model.dem_res_real[variavel, cenario]


def mccormick_3_Lshed(model, variavel, cenario):
      return model.k_Lshed[variavel, cenario] <= model.dem_res_real[variavel, cenario] * model.z[cenario]

def mccormick_4_Lshed(model, variavel, cenario):
      return model.k_Lshed[variavel, cenario] <= model.L_shed[variavel, cenario] + model.L_shed_min[variavel] * model.z[cenario] - model.L_shed_min[variavel]

model.mccormick_1_Lshed = Constraint(model.loads, model.scen, rule=mccormick_1_Lshed)
model.mccormick_2_Lshed = Constraint(model.loads, model.scen, rule=mccormick_2_Lshed)
model.mccormick_3_Lshed = Constraint(model.loads, model.scen, rule=mccormick_3_Lshed)
model.mccormick_4_Lshed = Constraint(model.loads, model.scen, rule=mccormick_4_Lshed)

#Restrição confiança
def restricao_adicional_mccormick(model):
    return sum(model.z[cen] * model.prob[cen] for cen in model.scen) >= confianca
model.restricao_adicional = Constraint(rule=restricao_adicional_mccormick)

def operation_balance(model, carga, cen): #Restrição do balanço
    thermal_reserve_ativation = sum(model.k_r[g, cen] for g in model.ger)
    shed_load = sum(model.k_Lshed[l, cen] for l in model.loads)
    return thermal_reserve_ativation + shed_load == model.dem_res_real[carga,cen]*model.z[cen]
model.operation_balance = Constraint(model.loads, model.scen, rule=operation_balance)

# Declarar Função Objetivo
def FOB(model):
    #Parte Determinística
    custo_energia = sum(model.c_ener[g]*model.P_ger[g] for g in model.ger)
    custo_reserva = sum(model.c_res[g]*model.R[g] for g in model.ger)
    FOB_det = custo_energia + custo_reserva

    #Parte Probabilística
    FOB_prob = 0
    for cen in model.scen:
        custo_reserva_ativada = sum(model.c_ener[g]*model.r[g,cen] for g in model.ger)
        custo_deslastre = sum(model.c_shed[l]*model.L_shed[l,cen] for l in model.loads)
        FOB_prob += model.prob[cen] * (custo_reserva_ativada + custo_deslastre)
    return FOB_det + FOB_prob

model.objective = Objective(rule=FOB, sense=minimize)
solver = SolverFactory('ipopt')
resultados = solver.solve(model, tee=False)


####Resultados

In [58]:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo:', value(model.objective), '\n')

#Print das Variáveis
for i in model.ger:
    print(f'Produção do {i}:' ,value(model.P_ger[i]), '\n')
    print(f'Reserva do {i}:' ,value(model.R[i]), '\n')


for j in model.scen:
    for i in model.loads:
        print(f'Deslastre do {i} no {j}:', value(model.L_shed[i,j]), '\n')
    for i in model.ger:
        print(f'Reserva ativada do {i} no {j}:' ,value(model.r[i,j]), '\n')

#Variáveis de decisão Z:
for i in model.scen:
    print(f'Z_{i}:' ,value(model.z[i]), '\n')

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: 2194.9999787145725 

Produção do Produtor A: 90.00000100981073 

Reserva do Produtor A: 9.999999990063978 

Produção do Produtor B: 39.999998990189276 

Reserva do Produtor B: 0.0 

Deslastre do Consumidor 1 no Cenario_1: 0.0 

Reserva ativada do Produtor A no Cenario_1: 3.00159965964099e-09 

Reserva ativada do Produtor B no Cenario_1: 0.0 

Deslastre do Consumidor 1 no Cenario_2: 0.0 

Reserva ativada do Produtor A no Cenario_2: 9.999999885187991 

Reserva ativada do Produtor B no Cenario_2: 0.0 

Z_Cenario_1: 1.5429880757308098e-11 

Z_Cenario_2: 0.9999999894988796 



###Uma Eólica